# Sit746 Experiments

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#code for PER
import torch
from torch import nn, optim
import random
import time
import pandas as pd
from collections import deque
import torch.nn.functional as F


class PER:
    def __init__(self, capacity, alpha=0.6):
        self.capacity = capacity
        self.alpha = alpha
        self.buffer = []
        self.priorities = []
        self.position = 0

    def add(self, state, action,reward, next_state, done, n_steps_arr, error):
        priority = (abs(error)+1e-5)**self.alpha
        if len(self.buffer) < self.capacity:
            self.buffer.append((state,action,reward,next_state,done,n_steps_arr))
            self.priorities.append(priority)
        else:
            self.buffer[self.position] = (state,action,reward,next_state,done,n_steps_arr)
            self.priorities[self.position] = priority
        self.position = (self.position+1)%self.capacity
        
    def sample(self, batch_size, beta=0.4, device="cpu"):
        if len(self.buffer) < batch_size:
            return None  # Return None instead of doing nothing (prevents errors)
        probs = np.array(self.priorities)/sum(self.priorities)
        indices = np.random.choice(len(self.buffer), batch_size, p=probs)  
        batch = [self.buffer[i] for i in indices]  # Extract batch from buffer
    
        # Efficiently unpack and convert to tensors
        states, actions, rewards, next_states, dones, n_steps_arr = zip(*batch)
        weights = (len(self.buffer)*probs[indices])**(-beta)
        weights /= weights.max()
    
        return (
                torch.tensor(np.array(states), dtype=torch.float32, device=device),
                torch.tensor(np.array(actions), dtype=torch.long, device=device),
                torch.tensor(np.array(rewards), dtype=torch.float32, device=device),
                torch.tensor(np.array(next_states), dtype=torch.float32, device=device),
                torch.tensor(np.array(dones), dtype=torch.float32, device=device), 
                torch.tensor(np.array(n_steps_arr), dtype=torch.float32, device = device),
                torch.tensor(weights, dtype=torch.float32, device=device),
                indices
            )
    def update_priorities(self, indices, errors):
        for i, error in zip(indices, errors):
            self.priorities[i] = (abs(error)+1e-5)**self.alpha




Below we import all the train and test data sets for the main experiments and generalization crypto experiments.

In [ ]:
##importing yahoo datasets
import yfinance as yf
#apple train and validation
apple_train = yf.download('aapl', '2007-01-01','2017-01-01',multi_level_index=False,progress=False)
apple_valid = yf.download('aapl', '2018-01-01','2021-01-01',multi_level_index=False,progress=False)
#ge train and validation
ge_train = yf.download('ge', '2007-01-01','2017-01-01',multi_level_index=False,progress=False)
ge_valid = yf.download('ge', '2018-01-01','2021-01-01',multi_level_index=False,progress=False)

#DJI train and validation
dji_train = yf.download('^dji', '2007-01-01','2017-01-01',multi_level_index=False,progress=False)
dji_valid = yf.download('^dji', '2018-01-01','2021-01-01',multi_level_index=False,progress=False)


In [ ]:
#crypto btc
btc_train = yf.download('btc-usd', '2018-01-01','2024-01-01',multi_level_index=False,progress=False)
btc_valid = yf.download('btc-usd', '2024-01-01','2025-07-01',multi_level_index=False,progress=False)

#crypto eth
eth_train = yf.download('eth-usd', '2018-01-01','2024-01-01',multi_level_index=False,progress=False)
eth_valid = yf.download('eth-usd', '2024-01-01','2025-07-01',multi_level_index=False,progress=False)

#crypto sol, bnb
sol_valid = yf.download('sol-usd', '2024-01-01','2025-07-01',multi_level_index=False,progress=False)
bnb_valid = yf.download('bnb-usd', '2024-01-01','2025-07-01',multi_level_index=False,progress=False)
xrp_valid = yf.download('xrp-usd', '2024-01-01','2025-07-01',multi_level_index=False,progress=False)

In [ ]:
from pykalman import KalmanFilter
def kalman_filter(x, col):
    x = x.copy()
    
    for i in range(len(col)):
        prices = x[col[i]]
        y = prices.values
        
        #set up a simple random-walk + noise model
        kf = KalmanFilter(
            transition_matrices = [1],      # x_t = x_{t-1} + process_noise
            observation_matrices = [1],     # y_t = x_t + obs_noise
            transition_covariance = 0.01,   # tune: process noise variance
            observation_covariance = 1.0,   # tune: observation noise variance
            initial_state_mean = y[0],
            initial_state_covariance = 1.0,
            random_state = 42
        )
        
        #Run the filter + smoother
        state_means, state_covs = kf.smooth(y)
        
        #Put the smoothed values back into a Series
        x[f'kalman_{col[i]}'] = state_means.flatten()
    return x



In [ ]:
import pykalman
import matplotlib
print(np.__version__)
print(pd.__version__)
print(yf.__version__)
print(torch.__version__)
print(pykalman.__version__)
print(matplotlib.__version__)


Define a preprocess function that does all feature engineering. In this case it only applies Kalman filter, but in the future if there were extra steps the functions would all be added into this to streamline the process. 

In [ ]:
def preprocess(x):
    x = x.copy()
    cols = x.columns
    x = kalman_filter(x,cols)
    return x

In [ ]:
app_t_pre = preprocess(apple_train)
app_v_pre = preprocess(apple_valid)
ge_t_pre = preprocess(ge_train)
ge_v_pre = preprocess(ge_valid)
dji_t_pre = preprocess(dji_train)
dji_v_pre = preprocess(dji_valid)
btc_t_pre = preprocess(btc_train)
btc_v_pre = preprocess(btc_valid)
eth_v_pre = preprocess(eth_valid)
sol_v_pre = preprocess(sol_valid)
bnb_v_pre = preprocess(bnb_valid)
xrp_v_pre = preprocess(xrp_valid)


In [ ]:
#save all preprocessed data to csv files for future work incase Yahoo Finance is down
app_t_pre.to_csv('apple_train.csv') 
app_v_pre.to_csv('apple_valid.csv') 
ge_t_pre.to_csv('ge_train.csv')  
ge_v_pre.to_csv('ge_valid.csv')  
dji_t_pre.to_csv('dji_train.csv')  
dji_v_pre.to_csv('dji_valid.csv')  
btc_t_pre.to_csv('btc_train.csv')  
btc_v_pre.to_csv('btc_valid.csv')  
eth_v_pre.to_csv('eth_valid.csv')  
sol_v_pre.to_csv('sol_valid.csv')  
bnb_v_pre.to_csv('bnb_valid.csv')  
xrp_v_pre.to_csv('xrp_valid.csv')  


In [ ]:
import SingleStockKFohlcvCuilogr as envv2

In [ ]:
lb = 20 #window length
vars = 5 #number of variables OHLCV
trans = 0.001 #0.1% transaction cost
#create all validation environments for main and generalization experiments
env_v = envv2.SingleStockTraderKFohlcvCuilogr(btc_v_pre, lookback=lb, transaction = trans)
env_v1 = envv2.SingleStockTraderKFohlcvCuilogr(eth_v_pre, lookback=lb, transaction = trans)
env_v2 = envv2.SingleStockTraderKFohlcvCuilogr(sol_v_pre, lookback=lb, transaction = trans)
env_v3 = envv2.SingleStockTraderKFohlcvCuilogr(bnb_v_pre, lookback=lb, transaction = trans)
env_v4 = envv2.SingleStockTraderKFohlcvCuilogr(xrp_v_pre, lookback=lb, transaction = trans)
env_v5 = envv2.SingleStockTraderKFohlcvCuilogr(app_v_pre, lookback=lb, transaction = trans)
env_v6 = envv2.SingleStockTraderKFohlcvCuilogr(ge_v_pre, lookback=lb, transaction = trans)
env_v7 = envv2.SingleStockTraderKFohlcvCuilogr(dji_v_pre, lookback=lb, transaction = trans)


Below is the standardization function that subtracts mean and divides by standard deviation before the inputs go into the network. The 1e-8 term is to prevent division of 0.

In [ ]:
def norm(state):
    market = state.copy()
    market = (market-market.mean(axis=0))/(market.std(axis=0)+1e-8)
    return market.flatten()

Below is the dual CNN layers with C51. The multi-step learning and PER is built into the full algorithm after this block.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class C51CNN(nn.Module):
    def __init__(
        self,
        state_dim: int,
        action_dim: int,
        lb: int,
        num_vars: int,
        hidden: int = 64,
        hidden2: int = 32,
        num_atoms: int = 51,
        Vmin: float = -10.0,
        Vmax: float = 10.0,
        dropout_p: float = 0.1,
    ):
        super().__init__()
        self.lb = lb
        self.num_vars = num_vars
        self.action_dim = action_dim
        self.num_atoms = num_atoms
        self.Vmin, self.Vmax = Vmin, Vmax
        self.delta_z = (Vmax - Vmin) / (num_atoms - 1)
        self.register_buffer("support", torch.linspace(Vmin, Vmax, num_atoms))

        # 1) CNN encoder
        self.conv_net = nn.Sequential(
            nn.Conv1d(num_vars, hidden, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.Conv1d(hidden, hidden2, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(start_dim=1),
        )

        # 2) MLP head with dropout
        self.dropout = nn.Dropout(p=dropout_p)
        self.fc2 = nn.Linear(hidden2, action_dim * num_atoms)

    def forward(self, s: torch.Tensor):
        B = s.size(0)       

        # reshape market -> (B, num_vars, lb)
        mkt = s.view(B, self.lb, self.num_vars).permute(0, 2, 1)

        # 1) CNN encoder
        x = self.conv_net(mkt)  

        # 2) MLP head
        x = self.dropout(x)

        # 3) Distributional head
        logits = self.fc2(x).view(B, self.action_dim, self.num_atoms)
        probs = F.softmax(logits, dim=2)
        return probs


## Main Experiment

### 1. DJI experiment

In [ ]:
env = envv2.SingleStockTraderKFohlcvCuilogr(dji_t_pre, lookback=lb, transaction =trans) 

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from collections import deque
import random
import time


gamma = 0.95
action_dim = env.action_space.n
state_dim = lb*vars
memory = 25000
batch_size = 8
max_episodes = 200
max_steps = 315
n_steps = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
init_lr = 1e-3

epsilon = 1
epsilon_min = 0.1
epsilon_decay = 0.98

# C51-specific hyperparameters
num_atoms = 51
Vmin = -10
Vmax = 10

# Instantiate networks and optimizer
policy = C51CNN(state_dim, action_dim, lb, vars).to(device)
target = C51CNN(state_dim, action_dim, lb, vars).to(device)
target.load_state_dict(policy.state_dict())
optimizer = optim.Adam(policy.parameters(), lr=init_lr, weight_decay= 1e-6)

beta_start = 0.4
beta_end = 1
beta_frames = max_episodes * max_steps


replay = PER(memory, alpha=0.3)


def projection_distribution(next_distr, rewards, dones, n_steps_arr, gamma, support, num_atoms, Vmin, Vmax):

    batch_size = rewards.size(0)
    delta_z = (Vmax - Vmin) / (num_atoms - 1)
    multiplier = torch.pow(gamma, n_steps_arr.float()).unsqueeze(1)  # shape: [batch, 1]
    # Compute Tz = reward + (gamma^n)*support, zeroing out future rewards if terminal
    Tz = rewards.unsqueeze(1) + multiplier * support.unsqueeze(0) * (1 - dones.unsqueeze(1))
    Tz = Tz.clamp(Vmin, Vmax)
    b = (Tz - Vmin) / delta_z  # position of Tz in the support space
    l = b.floor().long()
    u = b.ceil().long()
    
    proj_dist = torch.zeros(batch_size, num_atoms, device=rewards.device)
    # Distribute probability mass to nearest bins
    for j in range(num_atoms):
        mass = next_distr[:, j]
        proj_dist.scatter_add_(1, l[:, j].unsqueeze(1), (mass * (u[:, j].float() - b[:, j])).unsqueeze(1))
        proj_dist.scatter_add_(1, u[:, j].unsqueeze(1), (mass * (b[:, j] - l[:, j].float())).unsqueeze(1))
    
    return proj_dist

start = time.time()  
total_rewards = []
losses = []
valid_avg = []

global_step = 0


for ep in range(max_episodes):
    e = time.time()
    state = env.reset(random_start=True, steps=max_steps)[0]
    state = norm(state)
    done = False
    ep_reward = 0
    multi_step_buffer = deque(maxlen=n_steps)
    
    
    for step in range(max_steps):
        global_step += 1
        #beta =1  
        beta = min(beta_end, beta_start + (1 - beta_start) * (global_step / beta_frames))
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
        
            with torch.no_grad():
                prob_dist = policy(state_tensor)            
                q_values = torch.sum(prob_dist * policy.support, dim=2)  # expected Q-values
                action = torch.argmax(q_values, dim=1).item()
                
      
        next_state, reward, done, _, info = env.step(action)
        next_state = norm(next_state)
        ep_reward += reward
            
        multi_step_buffer.append((state, action, reward, next_state, done))
        if len(multi_step_buffer) == n_steps:
            G = 0
            for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
                G += (gamma ** idx) * r
                
            first_state, first_action, _, _, _ = multi_step_buffer[0]
            last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
            with torch.no_grad():
                first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
                pred_prob = policy(first_state_tensor)
                q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
                
                last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
                next_prob = target(last_next_state_tensor)
                max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
                effective_gamma = gamma ** len(multi_step_buffer)
                target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
                multi_step_error = abs(q_val - target_multi)
            replay.add(first_state, first_action, G, last_next_state, last_done, len(multi_step_buffer), multi_step_error)
            multi_step_buffer.popleft()
        state = next_state
        

        batch = replay.sample(batch_size, beta, device=device)
        if batch is not None:
            states, actions, rewards, next_states, dones, n_steps_arr, weights, indices = batch
            
            with torch.no_grad():
                # Get next-state distributions solely from the target network
                next_distr = target(next_states)
                # Compute Q-values from the target network distributions
                next_q = torch.sum(next_distr * target.support, dim=2)  # [batch, action_dim]
                # Select the best next action using target network Q-values
                next_actions = torch.argmax(next_q, dim=1)
                # Get the corresponding distribution for each transition
                next_dist = next_distr[torch.arange(batch_size), next_actions]  # [batch, num_atoms]
                
                # Project the target distribution onto the fixed support
                target_dist = projection_distribution(next_dist, rewards, dones, n_steps_arr, gamma, target.support, num_atoms, Vmin, Vmax)
                target_dist = target_dist.clamp(min=1e-8)

            
            # Get current predicted distribution for the taken actions
            pred_distr = policy(states)  # [batch, action_dim, num_atoms]
            pred_distr = pred_distr.gather(1, actions.unsqueeze(1).unsqueeze(2).expand(-1, 1, num_atoms)).squeeze(1)
            pred_distr = pred_distr.clamp(min=1e-8)
            
            log_pred = torch.log(pred_distr)
            loss_per_sample = - (target_dist * log_pred).sum(dim=1)
            loss = (loss_per_sample * weights).mean()
            losses.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()        
            optimizer.step()
            replay.update_priorities(indices, loss_per_sample.detach().cpu().numpy())
    
    # Flush any remaining transitions in multi-step buffer
    while multi_step_buffer:
        n = len(multi_step_buffer)
        G = 0
        for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
            G += (gamma ** idx) * r
            
        first_state, first_action, _, _, _ = multi_step_buffer[0]
        last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
        with torch.no_grad():
            first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
            pred_prob = policy(first_state_tensor)
            q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
            
            last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
            next_prob = target(last_next_state_tensor)
            max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
            effective_gamma = gamma ** len(multi_step_buffer)
            target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
            multi_step_error = abs(q_val - target_multi)
    
        replay.add(first_state, first_action, G, last_next_state, last_done, n, multi_step_error)
        multi_step_buffer.popleft()
        
   
    if global_step % 2000 == 0:
        target.load_state_dict(policy.state_dict())
    
    epsilon = max(epsilon_min, epsilon*epsilon_decay)

    total_rewards.append(ep_reward)
    
   
    val_state = env_v7.reset(random_start=False)[0]
    val_state = norm(val_state)
    val_done = False
    val_ep_reward = 0
    while not val_done:
        val_state_tensor = torch.tensor(val_state, dtype=torch.float32, device=device).unsqueeze(0)
        with torch.no_grad():
            val_prob = policy(val_state_tensor)
            val_q = torch.sum(val_prob * policy.support, dim=2)
            val_action = torch.argmax(val_q, dim=1).item()
        val_next_state, val_reward, val_done, _, _ = env_v7.step(val_action)
        val_next_state = norm(val_next_state)
        val_ep_reward += val_reward
        val_state = val_next_state
    valid_avg.append(val_ep_reward)

        
    s = time.time()
    
    if ep % 10 == 0:
        print(f'Episode {ep}, Time {(s - e):.2f}s, Training Reward: {ep_reward:.2f}, ' +
              f'Avg Training Reward: {np.mean(total_rewards):.2f}')
        print(f' Validation Reward: {val_ep_reward:.2f}, ' +
              f'Avg Validation Reward: {np.mean(valid_avg):.2f}')
    if (ep + 1) % 25 == 0:
        fig, ax = plt.subplots(1, 2, figsize=(18, 4))
        ax[0].plot(total_rewards)
        ax[0].plot(pd.Series(total_rewards).rolling(30, min_periods=1).mean())
        ax[0].set_title('training reward')
        ax[1].plot(valid_avg)
        ax[1].plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean())
        ax[1].set_title('validation reward')
        plt.show()

    
    # every 25 episodes, save policy weights
    if (ep + 1) % 25 == 0:
        ckpt_path = f"dji_policy_ep{ep+1:03d}.pth"
        torch.save(policy.state_dict(), ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

    
end = time.time()        
print('completed in', end - start)


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(valid_avg, label='valid reward dji', alpha=0.5)
plt.plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean(), label = 'valid smoothed reward', c='black', alpha=0.8)
plt.legend()
plt.xlabel('Episode Number', fontsize = 16)
plt.ylabel('Episode Reward', fontsize = 16)
plt.savefig(f'dji_valid_graph.png', dpi=1000)
plt.show()

In [ ]:
policy = C51CNN(state_dim, action_dim,lb,vars).to(device)

# 2) Load the state dict
ckpt = torch.load("t11_policy_ep050.pth", map_location=device)
policy.load_state_dict(ckpt)

In [ ]:
from scipy.stats import t

#calculate confidence interval
def calc_ci(x,n=10):
    xbar = np.mean(x)
    sd = np.std(x)
    t_c = t.ppf(1-0.05/2,n-1)
    width = t_c*sd/np.sqrt(n)
    return xbar.round(2), width.round(2)


In [ ]:
#create CI based graph
def shaded_region(ports, bnh, final, ticker):
    '''
    ports are all the portfolios over the 10 runs
    bnh is the buy and hold portfolio series
    final are the final values of the algorithm portfolios from ports
    '''
    mean_port = (pd.DataFrame(ports).T).mean(axis=1)
    width = calc_ci(final)[1]
    lower = mean_port-width
    upper = mean_port+width
    ind = bnh.index
    plt.figure(figsize=(10,6))
    plt.plot(ind, mean_port, label=f'{ticker}',c='b')
    plt.plot(ind,lower,alpha=0)
    plt.plot(ind, upper,alpha=0)
    plt.fill_between(ind,lower,upper,alpha=0.1, color='blue')
    plt.plot(ind,bnh.values, label='bnh', c='orange')
    plt.xlabel('Test Set Date Range', fontsize=16)
    plt.ylabel('Portfolio Value', fontsize=16)
    plt.ticklabel_format(style='plain', axis='y')
    plt.legend(fontsize=16)
    plt.savefig(f'{ticker}_port_graph.png', dpi=1000)
    plt.show()


In [ ]:
#evaluation functions
def sharpe_ratio(port):
    rets = port.pct_change().dropna()
    sharpe = np.sqrt(252)*(rets.mean())/(rets.std())
    return round(sharpe,2)

def annual_return(port):
    growth = port.iloc[-1]/port.iloc[0]
    n = len(port)
    ar = growth**(365/n)-1
    return 100*round(ar,4)

def profit_return(x, x0=500000):
    profit = x.iloc[-1] - x0
    pr = 100*(profit/x0)
    return round(profit,2), round(pr,2)

In [ ]:
profit_dji = []
pr_dji = []
sr_dji = []
ar_dji= []
ports_dji = []
final_dji = []

for _ in range(10):
    # Reset environment
    state = env_v7.reset()[0]
    state = norm(state)
    
    done = False
    episode_reward = 0
    episode_history = []
    
    # Single episode run using C51
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        # Get the distribution output: shape [1, action_dim, num_atoms]
        prob_dist = policy(state_tensor)
        # Compute expected Q-values: sum(prob * support) over atoms
        q_values = torch.sum(prob_dist * policy.support, dim=2)
        # Choose the action with the highest expected Q-value
        action = torch.argmax(q_values, dim=1).item()
        
        next_state, reward, done, _, info = env_v7.step(action)
        next_state = norm(next_state)
        episode_reward += reward
        episode_history.append(info)
        state = next_state
    
  
    pr_v_dji = pd.DataFrame(episode_history)
    port_dji = pr_v_dji.Portfolio
    profit_d = profit_return(port_dji)[0]
    pr_d = profit_return(port_dji)[1]
    sr_d = sharpe_ratio(port_dji)
    ar_d = annual_return(port_dji)
    profit_dji.append(profit_d)
    pr_dji.append(pr_d)
    sr_dji.append(sr_d)
    ar_dji.append(ar_d)
    ports_dji.append(port_dji)
    final_dji.append(port_dji.iloc[-1])
    
bnhs_dji = (500000 * np.cumprod(1 + dji_valid[lb-1:].Close.pct_change().dropna()))
bnh_dji = bnhs_dji.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_dji-500000)
print('bnh sr',sharpe_ratio(bnhs_dji))
print('bnh ar', annual_return(bnhs_dji))

print('dji profit', calc_ci(profit_dji))
print('dji pr', calc_ci(pr_dji))
print('dji sr', calc_ci(sr_dji))
print('dji ar', calc_ci(ar_dji))


In [ ]:
shaded_region(ports_dji,bnhs_dji,final_dji,'dji')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
dji_trial_runs = pd.DataFrame(ports_dji).T
dji_trial_runs.to_csv('dji_trial_runs.csv')

### 2. Apple Experiment

In [ ]:
env = envv2.SingleStockTraderKFohlcvCuilogr(app_t_pre, lookback=lb, transaction =trans) 

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from collections import deque
import random
import time


gamma = 0.95
action_dim = env.action_space.n
state_dim = lb*vars
memory = 25000
batch_size = 8
max_episodes = 200
max_steps = 315
n_steps = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
init_lr = 1e-3

epsilon = 1
epsilon_min = 0.1
epsilon_decay = 0.98

# C51-specific hyperparameters
num_atoms = 51
Vmin = -10
Vmax = 10

# Instantiate networks and optimizer
policy = C51CNN(state_dim, action_dim, lb, vars).to(device)
target = C51CNN(state_dim, action_dim, lb, vars).to(device)
target.load_state_dict(policy.state_dict())
optimizer = optim.Adam(policy.parameters(), lr=init_lr, weight_decay= 1e-6)

# Scheduler for learning rate
# at the top
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=75, gamma=0.5)

beta_start = 0.4
beta_end = 1
beta_frames = max_episodes * max_steps


replay = PER(memory, alpha=0.3)


def projection_distribution(next_distr, rewards, dones, n_steps_arr, gamma, support, num_atoms, Vmin, Vmax):

    batch_size = rewards.size(0)
    delta_z = (Vmax - Vmin) / (num_atoms - 1)
    multiplier = torch.pow(gamma, n_steps_arr.float()).unsqueeze(1) 
    # Compute Tz = reward + (gamma^n)*support, zeroing out future rewards if terminal
    Tz = rewards.unsqueeze(1) + multiplier * support.unsqueeze(0) * (1 - dones.unsqueeze(1))
    Tz = Tz.clamp(Vmin, Vmax)
    b = (Tz - Vmin) / delta_z  # position of Tz in the support space
    l = b.floor().long()
    u = b.ceil().long()
    
    proj_dist = torch.zeros(batch_size, num_atoms, device=rewards.device)
    # Distribute probability mass to nearest bins
    for j in range(num_atoms):
        mass = next_distr[:, j]
        proj_dist.scatter_add_(1, l[:, j].unsqueeze(1), (mass * (u[:, j].float() - b[:, j])).unsqueeze(1))
        proj_dist.scatter_add_(1, u[:, j].unsqueeze(1), (mass * (b[:, j] - l[:, j].float())).unsqueeze(1))
    
    return proj_dist

start = time.time()  
total_rewards = []
losses = []
valid_avg = []

global_step = 0


for ep in range(max_episodes):
    e = time.time()
    state = env.reset(random_start=True, steps=max_steps)[0]
    state = norm(state)
    done = False
    ep_reward = 0
    multi_step_buffer = deque(maxlen=n_steps)
    
    
    for step in range(max_steps):
        global_step += 1
        #beta =1  
        beta = min(beta_end, beta_start + (1 - beta_start) * (global_step / beta_frames))
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
        
            with torch.no_grad():
                prob_dist = policy(state_tensor)            
                q_values = torch.sum(prob_dist * policy.support, dim=2)  # expected Q-values
                action = torch.argmax(q_values, dim=1).item()
                
      
        next_state, reward, done, _, info = env.step(action)
        next_state = norm(next_state)
        ep_reward += reward
            
        multi_step_buffer.append((state, action, reward, next_state, done))
        if len(multi_step_buffer) == n_steps:
            G = 0
            for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
                G += (gamma ** idx) * r
                
            first_state, first_action, _, _, _ = multi_step_buffer[0]
            last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
            with torch.no_grad():
                first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
                pred_prob = policy(first_state_tensor)
                q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
                
                last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
                next_prob = target(last_next_state_tensor)
                max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
                effective_gamma = gamma ** len(multi_step_buffer)
                target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
                multi_step_error = abs(q_val - target_multi)
            replay.add(first_state, first_action, G, last_next_state, last_done, len(multi_step_buffer), multi_step_error)
            multi_step_buffer.popleft()
        state = next_state
        

        batch = replay.sample(batch_size, beta, device=device)
        if batch is not None:
            states, actions, rewards, next_states, dones, n_steps_arr, weights, indices = batch
            
            with torch.no_grad():
                # Get next-state distributions solely from the target network
                next_distr = target(next_states)
                # Compute Q-values from the target network distributions
                next_q = torch.sum(next_distr * target.support, dim=2) 
                # Select the best next action using target network Q-values
                next_actions = torch.argmax(next_q, dim=1)
                # Get the corresponding distribution for each transition
                next_dist = next_distr[torch.arange(batch_size), next_actions]  # [batch, num_atoms]
                
                # Project the target distribution onto the fixed support
                target_dist = projection_distribution(next_dist, rewards, dones, n_steps_arr, gamma, target.support, num_atoms, Vmin, Vmax)
                target_dist = target_dist.clamp(min=1e-8)

            
            # Get current predicted distribution for the taken actions
            pred_distr = policy(states)  
            pred_distr = pred_distr.gather(1, actions.unsqueeze(1).unsqueeze(2).expand(-1, 1, num_atoms)).squeeze(1)
            pred_distr = pred_distr.clamp(min=1e-8)
            
            log_pred = torch.log(pred_distr)
            loss_per_sample = - (target_dist * log_pred).sum(dim=1)
            loss = (loss_per_sample * weights).mean()
            losses.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()        
            optimizer.step()
            replay.update_priorities(indices, loss_per_sample.detach().cpu().numpy())
    
    # Flush any remaining transitions in multi-step buffer
    while multi_step_buffer:
        n = len(multi_step_buffer)
        G = 0
        for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
            G += (gamma ** idx) * r
            
        first_state, first_action, _, _, _ = multi_step_buffer[0]
        last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
        with torch.no_grad():
            first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
            pred_prob = policy(first_state_tensor)
            q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
            
            last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
            next_prob = target(last_next_state_tensor)
            max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
            effective_gamma = gamma ** len(multi_step_buffer)
            target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
            multi_step_error = abs(q_val - target_multi)
    
        replay.add(first_state, first_action, G, last_next_state, last_done, n, multi_step_error)
        multi_step_buffer.popleft()
        
   
    if global_step % 2000 == 0:
        target.load_state_dict(policy.state_dict())
    
    epsilon = max(epsilon_min, epsilon*epsilon_decay)

    total_rewards.append(ep_reward)
    
   
    val_state = env_v5.reset(random_start=False)[0]
    val_state = norm(val_state)
    val_done = False
    val_ep_reward = 0
    while not val_done:
        val_state_tensor = torch.tensor(val_state, dtype=torch.float32, device=device).unsqueeze(0)
        with torch.no_grad():
            val_prob = policy(val_state_tensor)
            val_q = torch.sum(val_prob * policy.support, dim=2)
            val_action = torch.argmax(val_q, dim=1).item()
        val_next_state, val_reward, val_done, _, _ = env_v5.step(val_action)
        val_next_state = norm(val_next_state)
        val_ep_reward += val_reward
        val_state = val_next_state
    valid_avg.append(val_ep_reward)

        
    s = time.time()
    
    if ep % 10 == 0:
        print(f'Episode {ep}, Time {(s - e):.2f}s, Training Reward: {ep_reward:.2f}, ' +
              f'Avg Training Reward: {np.mean(total_rewards):.2f}')
        print(f' Validation Reward: {val_ep_reward:.2f}, ' +
              f'Avg Validation Reward: {np.mean(valid_avg):.2f}')
    if (ep + 1) % 25 == 0:
        fig, ax = plt.subplots(1, 2, figsize=(18, 4))
        ax[0].plot(total_rewards)
        ax[0].plot(pd.Series(total_rewards).rolling(30, min_periods=1).mean())
        ax[0].set_title('training reward')
        ax[1].plot(valid_avg)
        ax[1].plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean())
        ax[1].set_title('validation reward')
        plt.show()

    
    # every 25 episodes, save policy weights
    if (ep + 1) % 25 == 0:
        ckpt_path = f"app_policy_ep{ep+1:03d}.pth"
        torch.save(policy.state_dict(), ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

    
end = time.time()        
print('completed in', end - start)


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(valid_avg, label='valid reward aapl', alpha=0.5)
plt.plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean(), label = 'valid smoothed reward', c='black', alpha=0.8)
plt.legend()
plt.xlabel('Episode Number', fontsize = 16)
plt.ylabel('Episode Reward', fontsize = 16)
plt.savefig(f'aapl_valid_graph.png', dpi=1000)
plt.show()

In [ ]:
profit_app = []
pr_app = []
sr_app = []
ar_app = []
ports_app = []
final_app = []

for _ in range(10):
    # Reset environment
    state = env_v5.reset()[0]
    state = norm(state)
    
    done = False
    episode_reward = 0
    episode_history = []
    
    # Single episode run using C51
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        # Get the distribution output: shape [1, action_dim, num_atoms]
        prob_dist = policy(state_tensor)
        # Compute expected Q-values: sum(prob * support) over atoms
        q_values = torch.sum(prob_dist * policy.support, dim=2)
        # Choose the action with the highest expected Q-value
        action = torch.argmax(q_values, dim=1).item()
        
        next_state, reward, done, _, info = env_v5.step(action)
        next_state = norm(next_state)
        episode_reward += reward
        episode_history.append(info)
        state = next_state
    
  
    pr_v_app = pd.DataFrame(episode_history)
    port_app = pr_v_app.Portfolio
    profit_a = profit_return(port_app)[0]
    pr_a = profit_return(port_app)[1]
    sr_a = sharpe_ratio(port_app)
    ar_a = annual_return(port_app)
    profit_app.append(profit_a)
    pr_app.append(pr_a)
    sr_app.append(sr_a)
    ar_app.append(ar_a)
    ports_app.append(port_app)
    final_app.append(port_app.iloc[-1])
    
bnhs_app = (500000 * np.cumprod(1 + apple_valid[lb-1:].Close.pct_change().dropna()))
bnh_app = bnhs_app.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_app-500000)
print('bnh sr',sharpe_ratio(bnhs_app))
print('bnh ar', annual_return(bnhs_app))

print('apple profit', calc_ci(profit_app))
print('apple pr', calc_ci(pr_app))
print('apple sr', calc_ci(sr_app))
print('apple ar', calc_ci(ar_app))

shaded_region(ports_app,bnhs_app,final_app,'aapl')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
app_trial_runs = pd.DataFrame(ports_app).T
app_trial_runs.to_csv('app_trial_runs.csv')

### 3. GE Experiment

In [ ]:
env = envv2.SingleStockTraderKFohlcvCuilogr(ge_t_pre, lookback=lb, transaction =trans) 

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from collections import deque
import random
import time


gamma = 0.95
action_dim = env.action_space.n
state_dim = lb*vars
memory = 25000
batch_size = 8
max_episodes = 200
max_steps = 315
n_steps = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
init_lr = 1e-3

epsilon = 1
epsilon_min = 0.1
epsilon_decay = 0.98

# C51-specific hyperparameters
num_atoms = 51
Vmin = -10
Vmax = 10

# Instantiate networks and optimizer
policy = C51CNN(state_dim, action_dim, lb, vars).to(device)
target = C51CNN(state_dim, action_dim, lb, vars).to(device)
target.load_state_dict(policy.state_dict())
optimizer = optim.Adam(policy.parameters(), lr=init_lr, weight_decay= 1e-6)


beta_start = 0.4
beta_end = 1
beta_frames = max_episodes * max_steps


replay = PER(memory, alpha=0.3)


def projection_distribution(next_distr, rewards, dones, n_steps_arr, gamma, support, num_atoms, Vmin, Vmax):

    batch_size = rewards.size(0)
    delta_z = (Vmax - Vmin) / (num_atoms - 1)
    multiplier = torch.pow(gamma, n_steps_arr.float()).unsqueeze(1)  
    # Compute Tz = reward + (gamma^n)*support, zeroing out future rewards if terminal
    Tz = rewards.unsqueeze(1) + multiplier * support.unsqueeze(0) * (1 - dones.unsqueeze(1))
    Tz = Tz.clamp(Vmin, Vmax)
    b = (Tz - Vmin) / delta_z  # position of Tz in the support space
    l = b.floor().long()
    u = b.ceil().long()
    
    proj_dist = torch.zeros(batch_size, num_atoms, device=rewards.device)
    # Distribute probability mass to nearest bins
    for j in range(num_atoms):
        mass = next_distr[:, j]
        proj_dist.scatter_add_(1, l[:, j].unsqueeze(1), (mass * (u[:, j].float() - b[:, j])).unsqueeze(1))
        proj_dist.scatter_add_(1, u[:, j].unsqueeze(1), (mass * (b[:, j] - l[:, j].float())).unsqueeze(1))
    
    return proj_dist

start = time.time()  
total_rewards = []
losses = []
valid_avg = []

global_step = 0


for ep in range(max_episodes):
    e = time.time()
    state = env.reset(random_start=True, steps=max_steps)[0]
    state = norm(state)
    done = False
    ep_reward = 0
    multi_step_buffer = deque(maxlen=n_steps)
    
    
    for step in range(max_steps):
        global_step += 1
        #beta =1  
        beta = min(beta_end, beta_start + (1 - beta_start) * (global_step / beta_frames))
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
        
            with torch.no_grad():
                prob_dist = policy(state_tensor)            
                q_values = torch.sum(prob_dist * policy.support, dim=2)  # expected Q-values
                action = torch.argmax(q_values, dim=1).item()
                
      
        next_state, reward, done, _, info = env.step(action)
        next_state = norm(next_state)
        ep_reward += reward
            
        multi_step_buffer.append((state, action, reward, next_state, done))
        if len(multi_step_buffer) == n_steps:
            G = 0
            for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
                G += (gamma ** idx) * r
                
            first_state, first_action, _, _, _ = multi_step_buffer[0]
            last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
            with torch.no_grad():
                first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
                pred_prob = policy(first_state_tensor)
                q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
                
                last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
                next_prob = target(last_next_state_tensor)
                max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
                effective_gamma = gamma ** len(multi_step_buffer)
                target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
                multi_step_error = abs(q_val - target_multi)
            replay.add(first_state, first_action, G, last_next_state, last_done, len(multi_step_buffer), multi_step_error)
            multi_step_buffer.popleft()
        state = next_state
        

        batch = replay.sample(batch_size, beta, device=device)
        if batch is not None:
            states, actions, rewards, next_states, dones, n_steps_arr, weights, indices = batch
            
            with torch.no_grad():
                # Get next-state distributions solely from the target network
                next_distr = target(next_states)
                # Compute Q-values from the target network distributions
                next_q = torch.sum(next_distr * target.support, dim=2)  
                # Select the best next action using target network Q-values
                next_actions = torch.argmax(next_q, dim=1)
                # Get the corresponding distribution for each transition
                next_dist = next_distr[torch.arange(batch_size), next_actions] 
                
                # Project the target distribution onto the fixed support
                target_dist = projection_distribution(next_dist, rewards, dones, n_steps_arr, gamma, target.support, num_atoms, Vmin, Vmax)
                target_dist = target_dist.clamp(min=1e-8)

            
            # Get current predicted distribution for the taken actions
            pred_distr = policy(states)  # [batch, action_dim, num_atoms]
            pred_distr = pred_distr.gather(1, actions.unsqueeze(1).unsqueeze(2).expand(-1, 1, num_atoms)).squeeze(1)
            pred_distr = pred_distr.clamp(min=1e-8)
            
            log_pred = torch.log(pred_distr)
            loss_per_sample = - (target_dist * log_pred).sum(dim=1)
            loss = (loss_per_sample * weights).mean()
            losses.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()        
           
            optimizer.step()
            replay.update_priorities(indices, loss_per_sample.detach().cpu().numpy())
    
    # Flush any remaining transitions in multi-step buffer
    while multi_step_buffer:
        n = len(multi_step_buffer)
        G = 0
        for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
            G += (gamma ** idx) * r
            
        first_state, first_action, _, _, _ = multi_step_buffer[0]
        last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
        with torch.no_grad():
            first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
            pred_prob = policy(first_state_tensor)
            q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
            
            last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
            next_prob = target(last_next_state_tensor)
            max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
            effective_gamma = gamma ** len(multi_step_buffer)
            target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
            multi_step_error = abs(q_val - target_multi)
    
        replay.add(first_state, first_action, G, last_next_state, last_done, n, multi_step_error)
        multi_step_buffer.popleft()
        
   
    if global_step % 2000 == 0:
        target.load_state_dict(policy.state_dict())
    
    epsilon = max(epsilon_min, epsilon*epsilon_decay)

    total_rewards.append(ep_reward)
    
   
    val_state = env_v6.reset(random_start=False)[0]
    val_state = norm(val_state)
    val_done = False
    val_ep_reward = 0
    while not val_done:
        val_state_tensor = torch.tensor(val_state, dtype=torch.float32, device=device).unsqueeze(0)
        with torch.no_grad():
            val_prob = policy(val_state_tensor)
            val_q = torch.sum(val_prob * policy.support, dim=2)
            val_action = torch.argmax(val_q, dim=1).item()
        val_next_state, val_reward, val_done, _, _ = env_v6.step(val_action)
        val_next_state = norm(val_next_state)
        val_ep_reward += val_reward
        val_state = val_next_state
    valid_avg.append(val_ep_reward)

        
    s = time.time()
    
    if ep % 10 == 0:
        print(f'Episode {ep}, Time {(s - e):.2f}s, Training Reward: {ep_reward:.2f}, ' +
              f'Avg Training Reward: {np.mean(total_rewards):.2f}')
        print(f' Validation Reward: {val_ep_reward:.2f}, ' +
              f'Avg Validation Reward: {np.mean(valid_avg):.2f}')
    if (ep + 1) % 25 == 0:
        fig, ax = plt.subplots(1, 2, figsize=(18, 4))
        ax[0].plot(total_rewards)
        ax[0].plot(pd.Series(total_rewards).rolling(30, min_periods=1).mean())
        ax[0].set_title('training reward')
        ax[1].plot(valid_avg)
        ax[1].plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean())
        ax[1].set_title('validation reward')
        plt.show()

    
    # every 25 episodes, save policy weights
    if (ep + 1) % 25 == 0:
        ckpt_path = f"ge_policy_ep{ep+1:03d}.pth"
        torch.save(policy.state_dict(), ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

    
end = time.time()        
print('completed in', end - start)


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(valid_avg, label='valid reward ge', alpha=0.5)
plt.plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean(), label = 'valid smoothed reward', c='black', alpha=0.8)
plt.legend()
plt.xlabel('Episode Number', fontsize = 16)
plt.ylabel('Episode Reward', fontsize = 16)
plt.savefig(f'ge_valid_graph.png', dpi=1000)
plt.show()

In [ ]:
profit_ge = []
pr_ge = []
sr_ge = []
ar_ge= []
ports_ge = []
final_ge = []

for _ in range(10):
    # Reset environment
    state = env_v6.reset()[0]
    state = norm(state)
    
    done = False
    episode_reward = 0
    episode_history = []
    
    # Single episode run using C51
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        # Get the distribution output: shape [1, action_dim, num_atoms]
        prob_dist = policy(state_tensor)
        # Compute expected Q-values: sum(prob * support) over atoms
        q_values = torch.sum(prob_dist * policy.support, dim=2)
        # Choose the action with the highest expected Q-value
        action = torch.argmax(q_values, dim=1).item()
        
        next_state, reward, done, _, info = env_v6.step(action)
        next_state = norm(next_state)
        episode_reward += reward
        episode_history.append(info)
        state = next_state
    
  
    pr_v_ge = pd.DataFrame(episode_history)
    port_ge = pr_v_ge.Portfolio
    profit_g = profit_return(port_ge)[0]
    pr_g = profit_return(port_ge)[1]
    sr_g = sharpe_ratio(port_ge)
    ar_g = annual_return(port_ge)
    profit_ge.append(profit_g)
    pr_ge.append(pr_g)
    sr_ge.append(sr_g)
    ar_ge.append(ar_g)
    ports_ge.append(port_ge)
    final_ge.append(port_ge.iloc[-1])
    
bnhs_ge = (500000 * np.cumprod(1 + ge_valid[lb-1:].Close.pct_change().dropna()))
bnh_ge = bnhs_ge.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_ge-500000)
print('bnh sr',sharpe_ratio(bnhs_ge))
print('bnh ar', annual_return(bnhs_ge))

print('ge profit', calc_ci(profit_ge))
print('ge pr', calc_ci(pr_ge))
print('ge sr', calc_ci(sr_ge))
print('ge ar', calc_ci(ar_ge))

shaded_region(ports_ge,bnhs_ge,final_ge,'ge')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
ge_trial_runs = pd.DataFrame(ports_ge).T
ge_trial_runs.to_csv('ge_trial_runs.csv')

## Generalization Experiments

In [ ]:
env = envv2.SingleStockTraderKFohlcvCuilogr(btc_t_pre, lookback=lb, transaction =trans)

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from collections import deque
import random
import time


gamma = 0.95
action_dim = env.action_space.n
state_dim = lb*vars
memory = 25000
batch_size = 8
max_episodes = 200
max_steps = 315
n_steps = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
init_lr = 1e-3

epsilon = 1
epsilon_min = 0.1
epsilon_decay = 0.98

# C51-specific hyperparameters
num_atoms = 51
Vmin = -10
Vmax = 10

# Instantiate networks and optimizer
policy = C51CNN(state_dim, action_dim, lb, vars).to(device)
target = C51CNN(state_dim, action_dim, lb, vars).to(device)
target.load_state_dict(policy.state_dict())
optimizer = optim.Adam(policy.parameters(), lr=init_lr, weight_decay= 1e-6)

beta_start = 0.4
beta_end = 1
beta_frames = max_episodes * max_steps


replay = PER(memory, alpha=0.3)


def projection_distribution(next_distr, rewards, dones, n_steps_arr, gamma, support, num_atoms, Vmin, Vmax):

    batch_size = rewards.size(0)
    delta_z = (Vmax - Vmin) / (num_atoms - 1)
    multiplier = torch.pow(gamma, n_steps_arr.float()).unsqueeze(1)  
    # Compute Tz = reward + (gamma^n)*support, zeroing out future rewards if terminal
    Tz = rewards.unsqueeze(1) + multiplier * support.unsqueeze(0) * (1 - dones.unsqueeze(1))
    Tz = Tz.clamp(Vmin, Vmax)
    b = (Tz - Vmin) / delta_z  # position of Tz in the support space
    l = b.floor().long()
    u = b.ceil().long()
    
    proj_dist = torch.zeros(batch_size, num_atoms, device=rewards.device)
    # Distribute probability mass to nearest bins
    for j in range(num_atoms):
        mass = next_distr[:, j]
        proj_dist.scatter_add_(1, l[:, j].unsqueeze(1), (mass * (u[:, j].float() - b[:, j])).unsqueeze(1))
        proj_dist.scatter_add_(1, u[:, j].unsqueeze(1), (mass * (b[:, j] - l[:, j].float())).unsqueeze(1))
    
    return proj_dist

start = time.time()  
total_rewards = []
losses = []
valid_avg = []

global_step = 0


for ep in range(max_episodes):
    e = time.time()
    state = env.reset(random_start=True, steps=max_steps)[0]
    state = norm(state)
    done = False
    ep_reward = 0
    multi_step_buffer = deque(maxlen=n_steps)
    
    
    for step in range(max_steps):
        global_step += 1
        #beta =1  
        beta = min(beta_end, beta_start + (1 - beta_start) * (global_step / beta_frames))
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
        
            with torch.no_grad():
                prob_dist = policy(state_tensor)            
                q_values = torch.sum(prob_dist * policy.support, dim=2)  
                action = torch.argmax(q_values, dim=1).item()
                
      
        next_state, reward, done, _, info = env.step(action)
        next_state = norm(next_state)
        ep_reward += reward
            
        multi_step_buffer.append((state, action, reward, next_state, done))
        if len(multi_step_buffer) == n_steps:
            G = 0
            for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
                G += (gamma ** idx) * r
                
            first_state, first_action, _, _, _ = multi_step_buffer[0]
            last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
            with torch.no_grad():
                first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
                pred_prob = policy(first_state_tensor)
                q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
                
                last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
                next_prob = target(last_next_state_tensor)
                max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
                effective_gamma = gamma ** len(multi_step_buffer)
                target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
                multi_step_error = abs(q_val - target_multi)
            replay.add(first_state, first_action, G, last_next_state, last_done, len(multi_step_buffer), multi_step_error)
            multi_step_buffer.popleft()
        state = next_state
        

        batch = replay.sample(batch_size, beta, device=device)
        if batch is not None:
            states, actions, rewards, next_states, dones, n_steps_arr, weights, indices = batch
            
            with torch.no_grad():
                # Get next-state distributions solely from the target network
                next_distr = target(next_states)
                # Compute Q-values from the target network distributions
                next_q = torch.sum(next_distr * target.support, dim=2) 
                # Select the best next action using target network Q-values
                next_actions = torch.argmax(next_q, dim=1)
                # Get the corresponding distribution for each transition
                next_dist = next_distr[torch.arange(batch_size), next_actions] 
                
                # Project the target distribution onto the fixed support
                target_dist = projection_distribution(next_dist, rewards, dones, n_steps_arr, gamma, target.support, num_atoms, Vmin, Vmax)
                target_dist = target_dist.clamp(min=1e-8)

            
            # Get current predicted distribution for the taken actions
            pred_distr = policy(states)  
            pred_distr = pred_distr.gather(1, actions.unsqueeze(1).unsqueeze(2).expand(-1, 1, num_atoms)).squeeze(1)
            pred_distr = pred_distr.clamp(min=1e-8)
            
            log_pred = torch.log(pred_distr)
            loss_per_sample = - (target_dist * log_pred).sum(dim=1)
            loss = (loss_per_sample * weights).mean()
            losses.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()        
            optimizer.step()
            replay.update_priorities(indices, loss_per_sample.detach().cpu().numpy())
    
    # Flush any remaining transitions in multi-step buffer
    while multi_step_buffer:
        n = len(multi_step_buffer)
        G = 0
        for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
            G += (gamma ** idx) * r
            
        first_state, first_action, _, _, _ = multi_step_buffer[0]
        last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
        with torch.no_grad():
            first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
            pred_prob = policy(first_state_tensor)
            q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
            
            last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
            next_prob = target(last_next_state_tensor)
            max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
            effective_gamma = gamma ** len(multi_step_buffer)
            target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
            multi_step_error = abs(q_val - target_multi)
    
        replay.add(first_state, first_action, G, last_next_state, last_done, n, multi_step_error)
        multi_step_buffer.popleft()
        
   
    if global_step % 2000 == 0:
        target.load_state_dict(policy.state_dict())
    
    epsilon = max(epsilon_min, epsilon*epsilon_decay)

    total_rewards.append(ep_reward)
    
   
    val_state = env_v.reset(random_start=False)[0]
    val_state = norm(val_state)
    val_done = False
    val_ep_reward = 0
    while not val_done:
        val_state_tensor = torch.tensor(val_state, dtype=torch.float32, device=device).unsqueeze(0)
        with torch.no_grad():
            val_prob = policy(val_state_tensor)
            val_q = torch.sum(val_prob * policy.support, dim=2)
            val_action = torch.argmax(val_q, dim=1).item()
        val_next_state, val_reward, val_done, _, _ = env_v.step(val_action)
        val_next_state = norm(val_next_state)
        val_ep_reward += val_reward
        val_state = val_next_state
    valid_avg.append(val_ep_reward)

        
    s = time.time()
    
    if ep % 10 == 0:
        print(f'Episode {ep}, Time {(s - e):.2f}s, Training Reward: {ep_reward:.2f}, ' +
              f'Avg Training Reward: {np.mean(total_rewards):.2f}')
        print(f' Validation Reward: {val_ep_reward:.2f}, ' +
              f'Avg Validation Reward: {np.mean(valid_avg):.2f}')
    if (ep + 1) % 25 == 0:
        fig, ax = plt.subplots(1, 2, figsize=(18, 4))
        ax[0].plot(total_rewards)
        ax[0].plot(pd.Series(total_rewards).rolling(30, min_periods=1).mean())
        ax[0].set_title('training reward')
        ax[1].plot(valid_avg)
        ax[1].plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean())
        ax[1].set_title('validation reward')
        plt.show()

    
    # every 25 episodes, save policy weights
    if (ep + 1) % 25 == 0:
        ckpt_path = f"btc_policy_ep{ep+1:03d}.pth"
        torch.save(policy.state_dict(), ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

    
end = time.time()        
print('completed in', end - start)


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(valid_avg, label='valid reward btc', alpha=0.5)
plt.plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean(), label = 'valid smoothed reward', c='black', alpha=0.8)
plt.legend()
plt.xlabel('Episode Number', fontsize = 16)
plt.ylabel('Episode Reward', fontsize = 16)
plt.savefig(f'btc_valid_graph.png', dpi=1000)
plt.show()

In [ ]:
profit_btc = []
pr_btc = []
sr_btc = []
ar_btc= []
ports_btc = []
final_btc = []

for _ in range(10):
    # Reset environment
    state = env_v.reset()[0]
    state = norm(state)
    
    done = False
    episode_reward = 0
    episode_history = []
    
    # Single episode run using C51
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        prob_dist = policy(state_tensor)
        # Compute expected Q-values: sum(prob * support) over atoms
        q_values = torch.sum(prob_dist * policy.support, dim=2)
        # Choose the action with the highest expected Q-value
        action = torch.argmax(q_values, dim=1).item()
        
        next_state, reward, done, _, info = env_v.step(action)
        next_state = norm(next_state)
        episode_reward += reward
        episode_history.append(info)
        state = next_state
    
  
    pr_v_btc = pd.DataFrame(episode_history)
    port_btc = pr_v_btc.Portfolio
    profit_b = profit_return(port_btc)[0]
    pr_b = profit_return(port_btc)[1]
    sr_b = sharpe_ratio(port_btc)
    ar_b = annual_return(port_btc)
    profit_btc.append(profit_b)
    pr_btc.append(pr_b)
    sr_btc.append(sr_b)
    ar_btc.append(ar_b)
    ports_btc.append(port_btc)
    final_btc.append(port_btc.iloc[-1])
    
bnhs_btc = (500000 * np.cumprod(1 + btc_valid[lb-1:].Close.pct_change().dropna()))
bnh_btc = bnhs_btc.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_btc-500000)
print('bnh sr',sharpe_ratio(bnhs_btc))
print('bnh ar', annual_return(bnhs_btc))

print('btc profit', calc_ci(profit_btc))
print('btc pr', calc_ci(pr_btc))
print('btc sr', calc_ci(sr_btc))
print('btc ar', calc_ci(ar_btc))

shaded_region(ports_btc,bnhs_btc,final_btc,'btc')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
btc_trial_runs = pd.DataFrame(ports_btc).T
btc_trial_runs.to_csv('btc_trial_runs.csv')

In [ ]:
profit_eth = []
pr_eth = []
sr_eth = []
ar_eth= []
ports_eth = []
final_eth = []

for _ in range(10):
    # Reset environment
    state = env_v1.reset()[0]
    state = norm(state)
    
    done = False
    episode_reward = 0
    episode_history = []
    
    # Single episode run using C51
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        prob_dist = policy(state_tensor)
        # Compute expected Q-values: sum(prob * support) over atoms
        q_values = torch.sum(prob_dist * policy.support, dim=2)
        # Choose the action with the highest expected Q-value
        action = torch.argmax(q_values, dim=1).item()
        
        next_state, reward, done, _, info = env_v1.step(action)
        next_state = norm(next_state)
        episode_reward += reward
        episode_history.append(info)
        state = next_state
    
  
    pr_v_eth = pd.DataFrame(episode_history)
    port_eth = pr_v_eth.Portfolio
    profit_e = profit_return(port_eth)[0]
    pr_e = profit_return(port_eth)[1]
    sr_e = sharpe_ratio(port_eth)
    ar_e = annual_return(port_eth)
    profit_eth.append(profit_e)
    pr_eth.append(pr_e)
    sr_eth.append(sr_e)
    ar_eth.append(ar_e)
    ports_eth.append(port_eth)
    final_eth.append(port_eth.iloc[-1])
    
bnhs_eth = (500000 * np.cumprod(1 + eth_valid[lb-1:].Close.pct_change().dropna()))
bnh_eth = bnhs_eth.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_eth-500000)
print('bnh sr',sharpe_ratio(bnhs_eth))
print('bnh ar', annual_return(bnhs_eth))

print('eth profit', calc_ci(profit_eth))
print('eth pr', calc_ci(pr_eth))
print('eth sr', calc_ci(sr_eth))
print('eth ar', calc_ci(ar_eth))

shaded_region(ports_eth,bnhs_eth,final_eth,'eth')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
eth_trial_runs = pd.DataFrame(ports_eth).T
eth_trial_runs.to_csv('eth_trial_runs.csv')

In [ ]:
profit_sol = []
pr_sol = []
sr_sol = []
ar_sol = []
ports_sol = []
final_sol = []

for _ in range(10):
    # Reset environment
    state = env_v2.reset()[0]
    state = norm(state)
    
    done = False
    episode_reward = 0
    episode_history = []
    
    # Single episode run using C51
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        prob_dist = policy(state_tensor)
        # Compute expected Q-values: sum(prob * support) over atoms
        q_values = torch.sum(prob_dist * policy.support, dim=2)
        # Choose the action with the highest expected Q-value
        action = torch.argmax(q_values, dim=1).item()
        
        next_state, reward, done, _, info = env_v2.step(action)
        next_state = norm(next_state)
        episode_reward += reward
        episode_history.append(info)
        state = next_state
    
  
    pr_v_sol = pd.DataFrame(episode_history)
    port_sol = pr_v_sol.Portfolio
    profit_s = profit_return(port_sol)[0]
    pr_s = profit_return(port_sol)[1]
    sr_s = sharpe_ratio(port_sol)
    ar_s = annual_return(port_sol)
    profit_sol.append(profit_s)
    pr_sol.append(pr_s)
    sr_sol.append(sr_s)
    ar_sol.append(ar_s)
    ports_sol.append(port_sol)
    final_sol.append(port_sol.iloc[-1])
    
bnhs_sol = (500000 * np.cumprod(1 + sol_valid[lb-1:].Close.pct_change().dropna()))
bnh_sol = bnhs_sol.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_sol-500000)
print('bnh sr',sharpe_ratio(bnhs_sol))
print('bnh ar', annual_return(bnhs_sol))

print('sol profit', calc_ci(profit_sol))
print('sol pr', calc_ci(pr_sol))
print('sol sr', calc_ci(sr_sol))
print('sol ar', calc_ci(ar_sol))

shaded_region(ports_sol,bnhs_sol,final_sol,'sol')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
sol_trial_runs = pd.DataFrame(ports_sol).T
sol_trial_runs.to_csv('sol_trial_runs.csv')

In [ ]:
profit_bnb = []
pr_bnb = []
sr_bnb = []
ar_bnb= []
ports_bnb = []
final_bnb = []

for _ in range(10):
    # Reset environment
    state = env_v3.reset()[0]
    state = norm(state)
    
    done = False
    episode_reward = 0
    episode_history = []
    
    # Single episode run using C51
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        prob_dist = policy(state_tensor)
        # Compute expected Q-values: sum(prob * support) over atoms
        q_values = torch.sum(prob_dist * policy.support, dim=2)
        # Choose the action with the highest expected Q-value
        action = torch.argmax(q_values, dim=1).item()
        
        next_state, reward, done, _, info = env_v3.step(action)
        next_state = norm(next_state)
        episode_reward += reward
        episode_history.append(info)
        state = next_state
    
  
    pr_v_bnb = pd.DataFrame(episode_history)
    port_bnb = pr_v_bnb.Portfolio
    profit_bn = profit_return(port_bnb)[0]
    pr_bn = profit_return(port_bnb)[1]
    sr_bn = sharpe_ratio(port_bnb)
    ar_bn = annual_return(port_bnb)
    profit_bnb.append(profit_bn)
    pr_bnb.append(pr_bn)
    sr_bnb.append(sr_bn)
    ar_bnb.append(ar_bn)
    ports_bnb.append(port_bnb)
    final_bnb.append(port_bnb.iloc[-1])
    
bnhs_bnb = (500000 * np.cumprod(1 + bnb_valid[lb-1:].Close.pct_change().dropna()))
bnh_bnb = bnhs_bnb.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_bnb-500000)
print('bnh sr',sharpe_ratio(bnhs_bnb))
print('bnh ar', annual_return(bnhs_bnb))

print('bnb profit', calc_ci(profit_bnb))
print('bnb pr', calc_ci(pr_bnb))
print('bnb sr', calc_ci(sr_bnb))
print('bnb ar', calc_ci(ar_bnb))

shaded_region(ports_bnb,bnhs_bnb,final_bnb,'bnb')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
bnb_trial_runs = pd.DataFrame(ports_bnb).T
bnb_trial_runs.to_csv('bnb_trial_runs.csv')

In [ ]:
profit_xrp = []
pr_xrp = []
sr_xrp = []
ar_xrp = []
ports_xrp = []
final_xrp = []

for _ in range(10):
    # Reset environment
    state = env_v4.reset()[0]
    state = norm(state)
    
    done = False
    episode_reward = 0
    episode_history = []
    
    # Single episode run using C51
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        prob_dist = policy(state_tensor)
        # Compute expected Q-values: sum(prob * support) over atoms
        q_values = torch.sum(prob_dist * policy.support, dim=2)
        # Choose the action with the highest expected Q-value
        action = torch.argmax(q_values, dim=1).item()
        
        next_state, reward, done, _, info = env_v4.step(action)
        next_state = norm(next_state)
        episode_reward += reward
        episode_history.append(info)
        state = next_state
    
  
    pr_v_xrp = pd.DataFrame(episode_history)
    port_xrp = pr_v_xrp.Portfolio
    profit_x = profit_return(port_xrp)[0]
    pr_x = profit_return(port_xrp)[1]
    sr_x = sharpe_ratio(port_xrp)
    ar_x = annual_return(port_xrp)
    profit_xrp.append(profit_x)
    pr_xrp.append(pr_x)
    sr_xrp.append(sr_x)
    ar_xrp.append(ar_x)
    ports_xrp.append(port_xrp)
    final_xrp.append(port_xrp.iloc[-1])
    
bnhs_xrp = (500000 * np.cumprod(1 + xrp_valid[lb-1:].Close.pct_change().dropna()))
bnh_xrp = bnhs_xrp.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_xrp-500000)
print('bnh sr',sharpe_ratio(bnhs_xrp))
print('bnh ar', annual_return(bnhs_xrp))

print('xrp profit', calc_ci(profit_xrp))
print('xrp pr', calc_ci(pr_xrp))
print('xrp sr', calc_ci(sr_xrp))
print('xrp ar', calc_ci(ar_xrp))

shaded_region(ports_xrp, bnhs_xrp, final_xrp,'xrp')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
xrp_trial_runs = pd.DataFrame(ports_xrp).T
xrp_trial_runs.to_csv('xrp_trial_runs.csv')

## Ablation Experiment

In [ ]:
import SingleStockohlcvCuilogr_no_kalman as envv2

In [ ]:
lb = 20 #window length
vars = 5 #number of variables OHLCV
trans = 0.001 #0.1% transaction cost
#create all validation environments for ablation experiments
env_v = envv2.SingleStockTraderohlcvCuilogr_no_kalman(app_v_pre, lookback=lb, transaction = trans)
env_v2 = envv2.SingleStockTraderohlcvCuilogr_no_kalman(ge_v_pre, lookback=lb, transaction = trans)
env_v3 = envv2.SingleStockTraderohlcvCuilogr_no_kalman(dji_v_pre, lookback=lb, transaction = trans)


### DJI no kalman filter

In [ ]:
env = envv2.SingleStockTraderohlcvCuilogr_no_kalman(dji_t_pre, lookback=lb, transaction = trans)

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from collections import deque
import random
import time


gamma = 0.95
action_dim = env.action_space.n
state_dim = lb*vars
memory = 25000
batch_size = 8
max_episodes = 200
max_steps = 315
n_steps = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
init_lr = 1e-3

epsilon = 1
epsilon_min = 0.1
epsilon_decay = 0.98

# C51-specific hyperparameters
num_atoms = 51
Vmin = -10
Vmax = 10

# Instantiate networks and optimizer
policy = C51CNN(state_dim, action_dim, lb, vars).to(device)
target = C51CNN(state_dim, action_dim, lb, vars).to(device)
target.load_state_dict(policy.state_dict())
optimizer = optim.Adam(policy.parameters(), lr=init_lr, weight_decay= 1e-6)

beta_start = 0.4
beta_end = 1
beta_frames = max_episodes * max_steps


replay = PER(memory, alpha=0.3)


def projection_distribution(next_distr, rewards, dones, n_steps_arr, gamma, support, num_atoms, Vmin, Vmax):

    batch_size = rewards.size(0)
    delta_z = (Vmax - Vmin) / (num_atoms - 1)
    multiplier = torch.pow(gamma, n_steps_arr.float()).unsqueeze(1)  
    # Compute Tz = reward + (gamma^n)*support, zeroing out future rewards if terminal
    Tz = rewards.unsqueeze(1) + multiplier * support.unsqueeze(0) * (1 - dones.unsqueeze(1))
    Tz = Tz.clamp(Vmin, Vmax)
    b = (Tz - Vmin) / delta_z  # position of Tz in the support space
    l = b.floor().long()
    u = b.ceil().long()
    
    proj_dist = torch.zeros(batch_size, num_atoms, device=rewards.device)
    # Distribute probability mass to nearest bins
    for j in range(num_atoms):
        mass = next_distr[:, j]
        proj_dist.scatter_add_(1, l[:, j].unsqueeze(1), (mass * (u[:, j].float() - b[:, j])).unsqueeze(1))
        proj_dist.scatter_add_(1, u[:, j].unsqueeze(1), (mass * (b[:, j] - l[:, j].float())).unsqueeze(1))
    
    return proj_dist

start = time.time()  
total_rewards = []
losses = []
valid_avg = []

global_step = 0


for ep in range(max_episodes):
    e = time.time()
    state = env.reset(random_start=True, steps=max_steps)[0]
    state = norm(state)
    done = False
    ep_reward = 0
    multi_step_buffer = deque(maxlen=n_steps)
    
    
    for step in range(max_steps):
        global_step += 1
        #beta =1  
        beta = min(beta_end, beta_start + (1 - beta_start) * (global_step / beta_frames))
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
        
            with torch.no_grad():
                prob_dist = policy(state_tensor)            
                q_values = torch.sum(prob_dist * policy.support, dim=2)  
                action = torch.argmax(q_values, dim=1).item()
                
      
        next_state, reward, done, _, info = env.step(action)
        next_state = norm(next_state)
        ep_reward += reward
            
        multi_step_buffer.append((state, action, reward, next_state, done))
        if len(multi_step_buffer) == n_steps:
            G = 0
            for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
                G += (gamma ** idx) * r
                
            first_state, first_action, _, _, _ = multi_step_buffer[0]
            last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
            with torch.no_grad():
                first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
                pred_prob = policy(first_state_tensor)
                q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
                
                last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
                next_prob = target(last_next_state_tensor)
                max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
                effective_gamma = gamma ** len(multi_step_buffer)
                target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
                multi_step_error = abs(q_val - target_multi)
            replay.add(first_state, first_action, G, last_next_state, last_done, len(multi_step_buffer), multi_step_error)
            multi_step_buffer.popleft()
        state = next_state
        

        batch = replay.sample(batch_size, beta, device=device)
        if batch is not None:
            states, actions, rewards, next_states, dones, n_steps_arr, weights, indices = batch
            
            with torch.no_grad():
                # Get next-state distributions solely from the target network
                next_distr = target(next_states)
                # Compute Q-values from the target network distributions
                next_q = torch.sum(next_distr * target.support, dim=2) 
                # Select the best next action using target network Q-values
                next_actions = torch.argmax(next_q, dim=1)
                # Get the corresponding distribution for each transition
                next_dist = next_distr[torch.arange(batch_size), next_actions] 
                
                # Project the target distribution onto the fixed support
                target_dist = projection_distribution(next_dist, rewards, dones, n_steps_arr, gamma, target.support, num_atoms, Vmin, Vmax)
                target_dist = target_dist.clamp(min=1e-8)

            
            # Get current predicted distribution for the taken actions
            pred_distr = policy(states)  
            pred_distr = pred_distr.gather(1, actions.unsqueeze(1).unsqueeze(2).expand(-1, 1, num_atoms)).squeeze(1)
            pred_distr = pred_distr.clamp(min=1e-8)
            
            log_pred = torch.log(pred_distr)
            loss_per_sample = - (target_dist * log_pred).sum(dim=1)
            loss = (loss_per_sample * weights).mean()
            losses.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()        
            optimizer.step()
            replay.update_priorities(indices, loss_per_sample.detach().cpu().numpy())
    
    # Flush any remaining transitions in multi-step buffer
    while multi_step_buffer:
        n = len(multi_step_buffer)
        G = 0
        for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
            G += (gamma ** idx) * r
            
        first_state, first_action, _, _, _ = multi_step_buffer[0]
        last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
        with torch.no_grad():
            first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
            pred_prob = policy(first_state_tensor)
            q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
            
            last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
            next_prob = target(last_next_state_tensor)
            max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
            effective_gamma = gamma ** len(multi_step_buffer)
            target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
            multi_step_error = abs(q_val - target_multi)
    
        replay.add(first_state, first_action, G, last_next_state, last_done, n, multi_step_error)
        multi_step_buffer.popleft()
        
   
    if global_step % 2000 == 0:
        target.load_state_dict(policy.state_dict())
    
    epsilon = max(epsilon_min, epsilon*epsilon_decay)

    total_rewards.append(ep_reward)
    
   
    val_state = env_v3.reset(random_start=False)[0]
    val_state = norm(val_state)
    val_done = False
    val_ep_reward = 0
    while not val_done:
        val_state_tensor = torch.tensor(val_state, dtype=torch.float32, device=device).unsqueeze(0)
        with torch.no_grad():
            val_prob = policy(val_state_tensor)
            val_q = torch.sum(val_prob * policy.support, dim=2)
            val_action = torch.argmax(val_q, dim=1).item()
        val_next_state, val_reward, val_done, _, _ = env_v3.step(val_action)
        val_next_state = norm(val_next_state)
        val_ep_reward += val_reward
        val_state = val_next_state
    valid_avg.append(val_ep_reward)

        
    s = time.time()
    
    if ep % 10 == 0:
        print(f'Episode {ep}, Time {(s - e):.2f}s, Training Reward: {ep_reward:.2f}, ' +
              f'Avg Training Reward: {np.mean(total_rewards):.2f}')
        print(f' Validation Reward: {val_ep_reward:.2f}, ' +
              f'Avg Validation Reward: {np.mean(valid_avg):.2f}')
    if (ep + 1) % 25 == 0:
        fig, ax = plt.subplots(1, 2, figsize=(18, 4))
        ax[0].plot(total_rewards)
        ax[0].plot(pd.Series(total_rewards).rolling(30, min_periods=1).mean())
        ax[0].set_title('training reward')
        ax[1].plot(valid_avg)
        ax[1].plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean())
        ax[1].set_title('validation reward')
        plt.show()

    
    # every 25 episodes, save policy weights
    if (ep + 1) % 25 == 0:
        ckpt_path = f"dji_no_kf_policy_ep{ep+1:03d}.pth"
        torch.save(policy.state_dict(), ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

    
end = time.time()        
print('completed in', end - start)


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(valid_avg, label='valid reward dji no KF', alpha=0.5)
plt.plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean(), label = 'valid smoothed reward', c='black', alpha=0.8)
plt.legend()
plt.xlabel('Episode Number', fontsize = 16)
plt.ylabel('Episode Reward', fontsize = 16)
plt.savefig(f'dji_no_kf_valid_graph.png', dpi=1000)
plt.show()

In [ ]:
profit_dji1 = []
pr_dji1 = []
sr_dji1 = []
ar_dji1 = []
ports_dji1 = []
final_dji1 = []

for _ in range(10):
    # Reset environment
    state = env_v3.reset()[0]
    state = norm(state)
    
    done = False
    episode_reward = 0
    episode_history = []
    
    # Single episode run using C51
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        # Get the distribution output: shape [1, action_dim, num_atoms]
        prob_dist = policy(state_tensor)
        # Compute expected Q-values: sum(prob * support) over atoms
        q_values = torch.sum(prob_dist * policy.support, dim=2)
        # Choose the action with the highest expected Q-value
        action = torch.argmax(q_values, dim=1).item()
        
        next_state, reward, done, _, info = env_v3.step(action)
        next_state = norm(next_state)
        episode_reward += reward
        episode_history.append(info)
        state = next_state
    
  
    pr_v_dji1 = pd.DataFrame(episode_history)
    port_dji1 = pr_v_dji1.Portfolio
    profit_d1 = profit_return(port_dji1)[0]
    pr_d1 = profit_return(port_dji1)[1]
    sr_d1 = sharpe_ratio(port_dji1)
    ar_d1 = annual_return(port_dji1)
    profit_dji1.append(profit_d1)
    pr_dji1.append(pr_d1)
    sr_dji1.append(sr_d1)
    ar_dji1.append(ar_d1)
    ports_dji1.append(port_dji1)
    final_dji1.append(port_dji1.iloc[-1])
    
bnhs_dji = (500000 * np.cumprod(1 + dji_valid[lb-1:].Close.pct_change().dropna()))
bnh_dji = bnhs_dji.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_dji-500000)
print('bnh sr',sharpe_ratio(bnhs_dji))
print('bnh ar', annual_return(bnhs_dji))

print('dji no KF profit', calc_ci(profit_dji1))
print('dji no KF pr', calc_ci(pr_dji1))
print('dji no KF sr', calc_ci(sr_dji1))
print('dji no KF ar', calc_ci(ar_dji1))
shaded_region(ports_dji1,bnhs_dji,final_dji1,'dji no KF')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
dji_no_kf_trial_runs = pd.DataFrame(ports_dji1).T
dji_no_kf_trial_runs.to_csv('dji_no_kf_trial_runs.csv')

### aaple no kf


In [ ]:
env = envv2.SingleStockTraderohlcvCuilogr_no_kalman(app_t_pre, lookback=lb, transaction = trans)

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from collections import deque
import random
import time


gamma = 0.95
action_dim = env.action_space.n
state_dim = lb*vars
memory = 25000
batch_size = 8
max_episodes = 200
max_steps = 315
n_steps = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
init_lr = 1e-3

epsilon = 1
epsilon_min = 0.1
epsilon_decay = 0.98

# C51-specific hyperparameters
num_atoms = 51
Vmin = -10
Vmax = 10

# Instantiate networks and optimizer
policy = C51CNN(state_dim, action_dim, lb, vars).to(device)
target = C51CNN(state_dim, action_dim, lb, vars).to(device)
target.load_state_dict(policy.state_dict())
optimizer = optim.Adam(policy.parameters(), lr=init_lr, weight_decay= 1e-6)

beta_start = 0.4
beta_end = 1
beta_frames = max_episodes * max_steps


replay = PER(memory, alpha=0.3)


def projection_distribution(next_distr, rewards, dones, n_steps_arr, gamma, support, num_atoms, Vmin, Vmax):

    batch_size = rewards.size(0)
    delta_z = (Vmax - Vmin) / (num_atoms - 1)
    multiplier = torch.pow(gamma, n_steps_arr.float()).unsqueeze(1)  
    # Compute Tz = reward + (gamma^n)*support, zeroing out future rewards if terminal
    Tz = rewards.unsqueeze(1) + multiplier * support.unsqueeze(0) * (1 - dones.unsqueeze(1))
    Tz = Tz.clamp(Vmin, Vmax)
    b = (Tz - Vmin) / delta_z  # position of Tz in the support space
    l = b.floor().long()
    u = b.ceil().long()
    
    proj_dist = torch.zeros(batch_size, num_atoms, device=rewards.device)
    # Distribute probability mass to nearest bins
    for j in range(num_atoms):
        mass = next_distr[:, j]
        proj_dist.scatter_add_(1, l[:, j].unsqueeze(1), (mass * (u[:, j].float() - b[:, j])).unsqueeze(1))
        proj_dist.scatter_add_(1, u[:, j].unsqueeze(1), (mass * (b[:, j] - l[:, j].float())).unsqueeze(1))
    
    return proj_dist

start = time.time()  
total_rewards = []
losses = []
valid_avg = []

global_step = 0


for ep in range(max_episodes):
    e = time.time()
    state = env.reset(random_start=True, steps=max_steps)[0]
    state = norm(state)
    done = False
    ep_reward = 0
    multi_step_buffer = deque(maxlen=n_steps)
    
    
    for step in range(max_steps):
        global_step += 1
        #beta =1  
        beta = min(beta_end, beta_start + (1 - beta_start) * (global_step / beta_frames))
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
        
            with torch.no_grad():
                prob_dist = policy(state_tensor)            
                q_values = torch.sum(prob_dist * policy.support, dim=2)  
                action = torch.argmax(q_values, dim=1).item()
                
      
        next_state, reward, done, _, info = env.step(action)
        next_state = norm(next_state)
        ep_reward += reward
            
        multi_step_buffer.append((state, action, reward, next_state, done))
        if len(multi_step_buffer) == n_steps:
            G = 0
            for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
                G += (gamma ** idx) * r
                
            first_state, first_action, _, _, _ = multi_step_buffer[0]
            last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
            with torch.no_grad():
                first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
                pred_prob = policy(first_state_tensor)
                q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
                
                last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
                next_prob = target(last_next_state_tensor)
                max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
                effective_gamma = gamma ** len(multi_step_buffer)
                target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
                multi_step_error = abs(q_val - target_multi)
            replay.add(first_state, first_action, G, last_next_state, last_done, len(multi_step_buffer), multi_step_error)
            multi_step_buffer.popleft()
        state = next_state
        

        batch = replay.sample(batch_size, beta, device=device)
        if batch is not None:
            states, actions, rewards, next_states, dones, n_steps_arr, weights, indices = batch
            
            with torch.no_grad():
                # Get next-state distributions solely from the target network
                next_distr = target(next_states)
                # Compute Q-values from the target network distributions
                next_q = torch.sum(next_distr * target.support, dim=2) 
                # Select the best next action using target network Q-values
                next_actions = torch.argmax(next_q, dim=1)
                # Get the corresponding distribution for each transition
                next_dist = next_distr[torch.arange(batch_size), next_actions] 
                
                # Project the target distribution onto the fixed support
                target_dist = projection_distribution(next_dist, rewards, dones, n_steps_arr, gamma, target.support, num_atoms, Vmin, Vmax)
                target_dist = target_dist.clamp(min=1e-8)

            
            # Get current predicted distribution for the taken actions
            pred_distr = policy(states)  
            pred_distr = pred_distr.gather(1, actions.unsqueeze(1).unsqueeze(2).expand(-1, 1, num_atoms)).squeeze(1)
            pred_distr = pred_distr.clamp(min=1e-8)
            
            log_pred = torch.log(pred_distr)
            loss_per_sample = - (target_dist * log_pred).sum(dim=1)
            loss = (loss_per_sample * weights).mean()
            losses.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()        
            optimizer.step()
            replay.update_priorities(indices, loss_per_sample.detach().cpu().numpy())
    
    # Flush any remaining transitions in multi-step buffer
    while multi_step_buffer:
        n = len(multi_step_buffer)
        G = 0
        for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
            G += (gamma ** idx) * r
            
        first_state, first_action, _, _, _ = multi_step_buffer[0]
        last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
        with torch.no_grad():
            first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
            pred_prob = policy(first_state_tensor)
            q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
            
            last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
            next_prob = target(last_next_state_tensor)
            max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
            effective_gamma = gamma ** len(multi_step_buffer)
            target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
            multi_step_error = abs(q_val - target_multi)
    
        replay.add(first_state, first_action, G, last_next_state, last_done, n, multi_step_error)
        multi_step_buffer.popleft()
        
   
    if global_step % 2000 == 0:
        target.load_state_dict(policy.state_dict())
    
    epsilon = max(epsilon_min, epsilon*epsilon_decay)

    total_rewards.append(ep_reward)
    
   
    val_state = env_v.reset(random_start=False)[0]
    val_state = norm(val_state)
    val_done = False
    val_ep_reward = 0
    while not val_done:
        val_state_tensor = torch.tensor(val_state, dtype=torch.float32, device=device).unsqueeze(0)
        with torch.no_grad():
            val_prob = policy(val_state_tensor)
            val_q = torch.sum(val_prob * policy.support, dim=2)
            val_action = torch.argmax(val_q, dim=1).item()
        val_next_state, val_reward, val_done, _, _ = env_v.step(val_action)
        val_next_state = norm(val_next_state)
        val_ep_reward += val_reward
        val_state = val_next_state
    valid_avg.append(val_ep_reward)

        
    s = time.time()
    
    if ep % 10 == 0:
        print(f'Episode {ep}, Time {(s - e):.2f}s, Training Reward: {ep_reward:.2f}, ' +
              f'Avg Training Reward: {np.mean(total_rewards):.2f}')
        print(f' Validation Reward: {val_ep_reward:.2f}, ' +
              f'Avg Validation Reward: {np.mean(valid_avg):.2f}')
    if (ep + 1) % 25 == 0:
        fig, ax = plt.subplots(1, 2, figsize=(18, 4))
        ax[0].plot(total_rewards)
        ax[0].plot(pd.Series(total_rewards).rolling(30, min_periods=1).mean())
        ax[0].set_title('training reward')
        ax[1].plot(valid_avg)
        ax[1].plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean())
        ax[1].set_title('validation reward')
        plt.show()

    
    # every 25 episodes, save policy weights
    if (ep + 1) % 25 == 0:
        ckpt_path = f"app_no_kf_policy_ep{ep+1:03d}.pth"
        torch.save(policy.state_dict(), ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

    
end = time.time()        
print('completed in', end - start)


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(valid_avg, label='valid reward apple no KF', alpha=0.5)
plt.plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean(), label = 'valid smoothed reward', c='black', alpha=0.8)
plt.legend()
plt.xlabel('Episode Number', fontsize = 16)
plt.ylabel('Episode Reward', fontsize = 16)
plt.savefig(f'apple_no_kf_valid_graph.png', dpi=1000)
plt.show()

In [ ]:
profit_app1 = []
pr_app1 = []
sr_app1 = []
ar_app1 = []
ports_app1 = []
final_app1 = []

for _ in range(10):
    # Reset environment
    state = env_v.reset()[0]
    state = norm(state)
    
    done = False
    episode_reward = 0
    episode_history = []
    
    # Single episode run using C51
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        # Get the distribution output: shape [1, action_dim, num_atoms]
        prob_dist = policy(state_tensor)
        # Compute expected Q-values: sum(prob * support) over atoms
        q_values = torch.sum(prob_dist * policy.support, dim=2)
        # Choose the action with the highest expected Q-value
        action = torch.argmax(q_values, dim=1).item()
        
        next_state, reward, done, _, info = env_v.step(action)
        next_state = norm(next_state)
        episode_reward += reward
        episode_history.append(info)
        state = next_state
    
  
    pr_v_app1 = pd.DataFrame(episode_history)
    port_app1 = pr_v_app1.Portfolio
    profit_a1 = profit_return(port_app1)[0]
    pr_a1 = profit_return(port_app1)[1]
    sr_a1 = sharpe_ratio(port_app1)
    ar_a1 = annual_return(port_app1)
    profit_app1.append(profit_a1)
    pr_app1.append(pr_a1)
    sr_app1.append(sr_a1)
    ar_app1.append(ar_a1)
    ports_app1.append(port_app1)
    final_app1.append(port_app1.iloc[-1])
    
bnhs_app = (500000 * np.cumprod(1 + apple_valid[lb-1:].Close.pct_change().dropna()))
bnh_app = bnhs_app.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_app-500000)
print('bnh sr',sharpe_ratio(bnhs_app))
print('bnh ar', annual_return(bnhs_app))

print('apple no kf profit', calc_ci(profit_app1))
print('apple no k pr', calc_ci(pr_app1))
print('apple no k sr', calc_ci(sr_app1))
print('apple no k ar', calc_ci(ar_app1))

shaded_region(ports_app1,bnhs_app,final_app1,'apple no k')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
app_no_kf_trial_runs = pd.DataFrame(ports_app1).T
app_no_kf_trial_runs.to_csv('app_no_kf_trial_runs.csv')

### GE no kf

In [ ]:
env = envv2.SingleStockTraderohlcvCuilogr_no_kalman(ge_t_pre, lookback=lb, transaction = trans)

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from collections import deque
import random
import time


gamma = 0.95
action_dim = env.action_space.n
state_dim = lb*vars
memory = 25000
batch_size = 8
max_episodes = 200
max_steps = 315
n_steps = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
init_lr = 1e-3

epsilon = 1
epsilon_min = 0.1
epsilon_decay = 0.98

# C51-specific hyperparameters
num_atoms = 51
Vmin = -10
Vmax = 10

# Instantiate networks and optimizer
policy = C51CNN(state_dim, action_dim, lb, vars).to(device)
target = C51CNN(state_dim, action_dim, lb, vars).to(device)
target.load_state_dict(policy.state_dict())
optimizer = optim.Adam(policy.parameters(), lr=init_lr, weight_decay= 1e-6)

beta_start = 0.4
beta_end = 1
beta_frames = max_episodes * max_steps


replay = PER(memory, alpha=0.3)


def projection_distribution(next_distr, rewards, dones, n_steps_arr, gamma, support, num_atoms, Vmin, Vmax):

    batch_size = rewards.size(0)
    delta_z = (Vmax - Vmin) / (num_atoms - 1)
    multiplier = torch.pow(gamma, n_steps_arr.float()).unsqueeze(1)  
    # Compute Tz = reward + (gamma^n)*support, zeroing out future rewards if terminal
    Tz = rewards.unsqueeze(1) + multiplier * support.unsqueeze(0) * (1 - dones.unsqueeze(1))
    Tz = Tz.clamp(Vmin, Vmax)
    b = (Tz - Vmin) / delta_z  # position of Tz in the support space
    l = b.floor().long()
    u = b.ceil().long()
    
    proj_dist = torch.zeros(batch_size, num_atoms, device=rewards.device)
    # Distribute probability mass to nearest bins
    for j in range(num_atoms):
        mass = next_distr[:, j]
        proj_dist.scatter_add_(1, l[:, j].unsqueeze(1), (mass * (u[:, j].float() - b[:, j])).unsqueeze(1))
        proj_dist.scatter_add_(1, u[:, j].unsqueeze(1), (mass * (b[:, j] - l[:, j].float())).unsqueeze(1))
    
    return proj_dist

start = time.time()  
total_rewards = []
losses = []
valid_avg = []

global_step = 0


for ep in range(max_episodes):
    e = time.time()
    state = env.reset(random_start=True, steps=max_steps)[0]
    state = norm(state)
    done = False
    ep_reward = 0
    multi_step_buffer = deque(maxlen=n_steps)
    
    
    for step in range(max_steps):
        global_step += 1
        #beta =1  
        beta = min(beta_end, beta_start + (1 - beta_start) * (global_step / beta_frames))
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
        
            with torch.no_grad():
                prob_dist = policy(state_tensor)            
                q_values = torch.sum(prob_dist * policy.support, dim=2)  
                action = torch.argmax(q_values, dim=1).item()
                
      
        next_state, reward, done, _, info = env.step(action)
        next_state = norm(next_state)
        ep_reward += reward
            
        multi_step_buffer.append((state, action, reward, next_state, done))
        if len(multi_step_buffer) == n_steps:
            G = 0
            for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
                G += (gamma ** idx) * r
                
            first_state, first_action, _, _, _ = multi_step_buffer[0]
            last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
            with torch.no_grad():
                first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
                pred_prob = policy(first_state_tensor)
                q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
                
                last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
                next_prob = target(last_next_state_tensor)
                max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
                effective_gamma = gamma ** len(multi_step_buffer)
                target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
                multi_step_error = abs(q_val - target_multi)
            replay.add(first_state, first_action, G, last_next_state, last_done, len(multi_step_buffer), multi_step_error)
            multi_step_buffer.popleft()
        state = next_state
        

        batch = replay.sample(batch_size, beta, device=device)
        if batch is not None:
            states, actions, rewards, next_states, dones, n_steps_arr, weights, indices = batch
            
            with torch.no_grad():
                # Get next-state distributions solely from the target network
                next_distr = target(next_states)
                # Compute Q-values from the target network distributions
                next_q = torch.sum(next_distr * target.support, dim=2) 
                # Select the best next action using target network Q-values
                next_actions = torch.argmax(next_q, dim=1)
                # Get the corresponding distribution for each transition
                next_dist = next_distr[torch.arange(batch_size), next_actions] 
                
                # Project the target distribution onto the fixed support
                target_dist = projection_distribution(next_dist, rewards, dones, n_steps_arr, gamma, target.support, num_atoms, Vmin, Vmax)
                target_dist = target_dist.clamp(min=1e-8)

            
            # Get current predicted distribution for the taken actions
            pred_distr = policy(states)  
            pred_distr = pred_distr.gather(1, actions.unsqueeze(1).unsqueeze(2).expand(-1, 1, num_atoms)).squeeze(1)
            pred_distr = pred_distr.clamp(min=1e-8)
            
            log_pred = torch.log(pred_distr)
            loss_per_sample = - (target_dist * log_pred).sum(dim=1)
            loss = (loss_per_sample * weights).mean()
            losses.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()        
            optimizer.step()
            replay.update_priorities(indices, loss_per_sample.detach().cpu().numpy())
    
    # Flush any remaining transitions in multi-step buffer
    while multi_step_buffer:
        n = len(multi_step_buffer)
        G = 0
        for idx, (_, _, r, _, _) in enumerate(multi_step_buffer):
            G += (gamma ** idx) * r
            
        first_state, first_action, _, _, _ = multi_step_buffer[0]
        last_next_state, last_done = multi_step_buffer[-1][3], multi_step_buffer[-1][4]
        with torch.no_grad():
            first_state_tensor = torch.tensor(first_state, dtype=torch.float32, device=device).unsqueeze(0)
            pred_prob = policy(first_state_tensor)
            q_val = torch.sum(pred_prob * policy.support, dim=2).squeeze(0)[first_action].item()
            
            last_next_state_tensor = torch.tensor(last_next_state, dtype=torch.float32, device=device).unsqueeze(0)
            next_prob = target(last_next_state_tensor)
            max_next_q = torch.sum(next_prob * target.support, dim=2).max(1)[0].item()
            effective_gamma = gamma ** len(multi_step_buffer)
            target_multi = G + effective_gamma * (1 - int(last_done)) * max_next_q
            multi_step_error = abs(q_val - target_multi)
    
        replay.add(first_state, first_action, G, last_next_state, last_done, n, multi_step_error)
        multi_step_buffer.popleft()
        
   
    if global_step % 2000 == 0:
        target.load_state_dict(policy.state_dict())
    
    epsilon = max(epsilon_min, epsilon*epsilon_decay)

    total_rewards.append(ep_reward)
    
   
    val_state = env_v2.reset(random_start=False)[0]
    val_state = norm(val_state)
    val_done = False
    val_ep_reward = 0
    while not val_done:
        val_state_tensor = torch.tensor(val_state, dtype=torch.float32, device=device).unsqueeze(0)
        with torch.no_grad():
            val_prob = policy(val_state_tensor)
            val_q = torch.sum(val_prob * policy.support, dim=2)
            val_action = torch.argmax(val_q, dim=1).item()
        val_next_state, val_reward, val_done, _, _ = env_v2.step(val_action)
        val_next_state = norm(val_next_state)
        val_ep_reward += val_reward
        val_state = val_next_state
    valid_avg.append(val_ep_reward)

        
    s = time.time()
    
    if ep % 10 == 0:
        print(f'Episode {ep}, Time {(s - e):.2f}s, Training Reward: {ep_reward:.2f}, ' +
              f'Avg Training Reward: {np.mean(total_rewards):.2f}')
        print(f' Validation Reward: {val_ep_reward:.2f}, ' +
              f'Avg Validation Reward: {np.mean(valid_avg):.2f}')
    if (ep + 1) % 25 == 0:
        fig, ax = plt.subplots(1, 2, figsize=(18, 4))
        ax[0].plot(total_rewards)
        ax[0].plot(pd.Series(total_rewards).rolling(30, min_periods=1).mean())
        ax[0].set_title('training reward')
        ax[1].plot(valid_avg)
        ax[1].plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean())
        ax[1].set_title('validation reward')
        plt.show()

    
    # every 25 episodes, save policy weights
    if (ep + 1) % 25 == 0:
        ckpt_path = f"ge_no_kf_policy_ep{ep+1:03d}.pth"
        torch.save(policy.state_dict(), ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

    
end = time.time()        
print('completed in', end - start)


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(valid_avg, label='valid reward ge no KF', alpha=0.5)
plt.plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean(), label = 'valid smoothed reward', c='black', alpha=0.8)
plt.legend()
plt.xlabel('Episode Number', fontsize = 16)
plt.ylabel('Episode Reward', fontsize = 16)
plt.savefig(f'ge_no_kf_valid_graph.png', dpi=1000)
plt.show()

In [ ]:
profit_ge1 = []
pr_ge1 = []
sr_ge1 = []
ar_ge1 = []
ports_ge1 = []
final_ge1 = []

for _ in range(10):
    # Reset environment
    state = env_v2.reset()[0]
    state = norm(state)
    
    done = False
    episode_reward = 0
    episode_history = []
    
    # Single episode run using C51
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        # Get the distribution output: shape [1, action_dim, num_atoms]
        prob_dist = policy(state_tensor)
        # Compute expected Q-values: sum(prob * support) over atoms
        q_values = torch.sum(prob_dist * policy.support, dim=2)
        # Choose the action with the highest expected Q-value
        action = torch.argmax(q_values, dim=1).item()
        
        next_state, reward, done, _, info = env_v2.step(action)
        next_state = norm(next_state)
        episode_reward += reward
        episode_history.append(info)
        state = next_state
    
  
    pr_v_ge1 = pd.DataFrame(episode_history)
    port_ge1 = pr_v_ge1.Portfolio
    profit_g1 = profit_return(port_ge1)[0]
    pr_g1 = profit_return(port_ge1)[1]
    sr_g1 = sharpe_ratio(port_ge1)
    ar_g1 = annual_return(port_ge1)
    profit_ge1.append(profit_g1)
    pr_ge1.append(pr_g1)
    sr_ge1.append(sr_g1)
    ar_ge1.append(ar_g1)
    ports_ge1.append(port_ge1)
    final_ge1.append(port_ge1.iloc[-1])
    
bnhs_ge = (500000 * np.cumprod(1 + ge_valid[lb-1:].Close.pct_change().dropna()))
bnh_ge = bnhs_ge.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_ge-500000)
print('bnh sr',sharpe_ratio(bnhs_ge))
print('bnh ar', annual_return(bnhs_ge))

print('ge no kf profit', calc_ci(profit_ge1))
print('ge no kf pr', calc_ci(pr_ge1))
print('ge no kf sr', calc_ci(sr_ge1))
print('ge no kf ar', calc_ci(ar_ge1))

shaded_region(ports_ge1,bnhs_ge, final_ge1,'ge no kf')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
ge_no_kf_trial_runs = pd.DataFrame(ports_ge1).T
ge_no_kf_trial_runs.to_csv('ge_no_kf_trial_runs.csv')

## Ablation 2 No MPC

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DQN_CNN(nn.Module):
    def __init__(
        self,
        state_dim: int,
        action_dim: int,
        lb: int,
        num_vars: int,
        hidden: int = 64,
        hidden2: int = 32,
        dropout_p: float = 0.1,
    ):
        super().__init__()
        self.lb = lb
        self.num_vars = num_vars
        self.action_dim = action_dim
    

        # 1) CNN encoder
        self.conv_net = nn.Sequential(
            nn.Conv1d(num_vars, hidden, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.Conv1d(hidden, hidden2, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(start_dim=1),
        )

        # 2) MLP head with dropout
        self.dropout = nn.Dropout(p=dropout_p)
        self.fc1 = nn.Linear(hidden2, hidden2)
        self.fc2 = nn.Linear(hidden2, action_dim)

    def forward(self, s: torch.Tensor):
        B = s.size(0)       

        # reshape market -> (B, num_vars, lb)
        mkt = s.view(B, self.lb, self.num_vars).permute(0, 2, 1)

        # 1) CNN encoder
        x = self.conv_net(mkt)  

        # 2) MLP head
        x = self.dropout(x)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)        
        return x


In [ ]:
import SingleStockKFohlcvCuilogr as envv2

In [ ]:
lb = 20 #window length
vars = 5 #number of variables OHLCV
trans = 0.001 #0.1% transaction cost
env_v = envv2.SingleStockTraderKFohlcvCuilogr(app_v_pre, lookback=lb, transaction = trans)
env_v2 = envv2.SingleStockTraderKFohlcvCuilogr(ge_v_pre, lookback=lb, transaction = trans)
env_v3 = envv2.SingleStockTraderKFohlcvCuilogr(dji_v_pre, lookback=lb, transaction = trans)

### Apple no MPC

In [ ]:
env = envv2.SingleStockTraderKFohlcvCuilogr(app_t_pre, lookback=lb, transaction =trans) 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import random
import numpy as np
import time

gamma = 0.95
action_dim = env.action_space.n
state_dim = lb * vars
buffer_capacity = 25000
batch_size = 8
max_episodes = 200
max_steps = 315
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
init_lr = 1e-3

epsilon = 1.0
epsilon_min = 0.1
epsilon_decay = 0.98

target_update_every = 2000  # steps

#Simple Replay Buffer
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        # store as numpy for memory efficiency
        self.buffer.append((
            np.array(state, dtype=np.float32),
            int(action),
            float(reward),
            np.array(next_state, dtype=np.float32),
            bool(done)
        ))

    def sample(self, batch_size, device):
        if len(self.buffer) < batch_size:
            return None
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.tensor(np.stack(states), dtype=torch.float32, device=device)
        actions = torch.tensor(actions, dtype=torch.long, device=device)
        rewards = torch.tensor(rewards, dtype=torch.float32, device=device)
        next_states = torch.tensor(np.stack(next_states), dtype=torch.float32, device=device)
        dones = torch.tensor(dones, dtype=torch.float32, device=device)
        return states, actions, rewards, next_states, dones

    def __len__(self):
        return len(self.buffer)


policy = DQN_CNN(state_dim, action_dim, lb, vars).to(device)
target = DQN_CNN(state_dim, action_dim, lb, vars).to(device)
target.load_state_dict(policy.state_dict())
optimizer = optim.Adam(policy.parameters(), lr=init_lr, weight_decay=1e-6)

replay = ReplayBuffer(buffer_capacity)


start = time.time()
total_rewards = []
valid_avg = []
losses = [] 
global_step = 0

for ep in range(max_episodes):
    e = time.time()
    state = env.reset(random_start=True, steps=max_steps)[0]
    state = norm(state)
    done = False
    ep_reward = 0

    for step in range(max_steps):
        global_step += 1

        # epsilon-greedy
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            with torch.no_grad():
                s = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
                q_vals = policy(s)  
                action = torch.argmax(q_vals, dim=1).item()

        next_state, reward, done, _, _ = env.step(action)
        next_state = norm(next_state)
        ep_reward += reward

        replay.push(state, action, reward, next_state, done)
        state = next_state

        # Sample and learn
        batch = replay.sample(batch_size, device)
        if batch is not None:
            states, actions, rewards, next_states, dones = batch

            # Current Q(s,a)
            q_pred = policy(states).gather(1, actions.unsqueeze(1)).squeeze(1)  

            # DDQN target:
            with torch.no_grad():
                next_q_policy = policy(next_states)                 
                next_actions = torch.argmax(next_q_policy, dim=1)   
                next_q_target = target(next_states)                 
                next_q = next_q_target.gather(1, next_actions.unsqueeze(1)).squeeze(1)  
                q_target = rewards + gamma * (1.0 - dones) * next_q

            
            loss = F.mse_loss(q_pred, q_target)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Periodic hard target update by steps
        if global_step % target_update_every == 0:
            target.load_state_dict(policy.state_dict())

        if done:
            break

    # epsilon decay per episode
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    total_rewards.append(ep_reward)

    #validation
    val_state = env_v.reset(random_start=False)[0]
    val_state = norm(val_state)
    val_done = False
    val_ep_reward = 0
    while not val_done:
        with torch.no_grad():
            s = torch.tensor(val_state, dtype=torch.float32, device=device).unsqueeze(0)
            q_vals = policy(s)
            val_action = torch.argmax(q_vals, dim=1).item()
        val_next_state, val_reward, val_done, _, _ = env_v.step(val_action)
        val_next_state = norm(val_next_state)
        val_ep_reward += val_reward
        val_state = val_next_state
    valid_avg.append(val_ep_reward)

    s = time.time()
    if ep % 10 == 0:
        print(f'Episode {ep}, Time {(s - e):.2f}s, '
              f'Train Reward: {ep_reward:.2f}, Avg Train: {np.mean(total_rewards):.2f}, '
              f'Val: {val_ep_reward:.2f}, Avg Val: {np.mean(valid_avg):.2f}')

    # Save every 25 episodes
    if (ep + 1) % 25 == 0:
        ckpt_path = f"app_ddqn_policy_ep{ep+1:03d}.pth"
        torch.save(policy.state_dict(), ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

    if (ep + 1) % 25 == 0:
    fig, ax = plt.subplots(1, 2, figsize=(18, 4))
    ax[0].plot(total_rewards)
    ax[0].plot(pd.Series(total_rewards).rolling(30, min_periods=1).mean())
    ax[0].set_title('training reward')
    ax[1].plot(valid_avg)
    ax[1].plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean())
    ax[1].set_title('validation reward')
    plt.show()

end = time.time()
print('completed in', end - start)


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(valid_avg, label='valid reward apple no MPC', alpha=0.5)
plt.plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean(), label = 'valid smoothed reward', c='black', alpha=0.8)
plt.legend()
plt.xlabel('Episode Number', fontsize = 16)
plt.ylabel('Episode Reward', fontsize = 16)
plt.savefig(f'apple_no_mpc_valid_graph.png', dpi=1000)
plt.show()

In [ ]:
profit_app2 = []
pr_app2 = []
sr_app2 = []
ar_app2 = []
ports_app2 = []
final_app2 = []

for _ in range(10):
    # Reset environment
    state = env_v.reset()[0]         
    state = norm(state)

    done = False
    episode_reward = 0
    episode_history = []

    while not done:
        with torch.no_grad():
            state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
            q_values = policy(state_tensor)              
            action = torch.argmax(q_values, dim=1).item()

        next_state, reward, done, _, info = env_v.step(action)
        next_state = norm(next_state)

        episode_reward += reward
        episode_history.append(info)
        state = next_state
    
    pr_v_app2 = pd.DataFrame(episode_history)
    port_app2 = pr_v_app2.Portfolio
    profit_a2 = profit_return(port_app2)[0]
    pr_a2 = profit_return(port_app2)[1]
    sr_a2 = sharpe_ratio(port_app2)
    ar_a2 = annual_return(port_app2)
    profit_app2.append(profit_a2)
    pr_app2.append(pr_a2)
    sr_app2.append(sr_a2)
    ar_app2.append(ar_a2)
    ports_app2.append(port_app2)
    final_app2.append(port_app2.iloc[-1])
    
bnhs_app = (500000 * np.cumprod(1 + apple_valid[lb-1:].Close.pct_change().dropna()))
bnh_app = bnhs_app.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_app-500000)
print('bnh sr',sharpe_ratio(bnhs_app))
print('bnh ar', annual_return(bnhs_app))

print('apple no MPC profit', calc_ci(profit_app2))
print('apple no MPC pr', calc_ci(pr_app2))
print('apple no MPC sr', calc_ci(sr_app2))
print('apple no MPC ar', calc_ci(ar_app2))

shaded_region(ports_app2,bnhs_app,final_app2,'apple no MPC')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
app_no_mpc_trial_runs = pd.DataFrame(ports_app2).T
app_no_mpc_trial_runs.to_csv('app_no_mpc_trial_runs.csv')

### GE no MPC

In [ ]:
env = envv2.SingleStockTraderKFohlcvCuilogr(ge_t_pre, lookback=lb, transaction =trans) 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import random
import numpy as np
import time

gamma = 0.95
action_dim = env.action_space.n
state_dim = lb * vars
buffer_capacity = 25000
batch_size = 8
max_episodes = 200
max_steps = 315
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
init_lr = 1e-3

epsilon = 1.0
epsilon_min = 0.1
epsilon_decay = 0.98

target_update_every = 2000  # steps

#Simple Replay Buffer
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        # store as numpy for memory efficiency
        self.buffer.append((
            np.array(state, dtype=np.float32),
            int(action),
            float(reward),
            np.array(next_state, dtype=np.float32),
            bool(done)
        ))

    def sample(self, batch_size, device):
        if len(self.buffer) < batch_size:
            return None
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.tensor(np.stack(states), dtype=torch.float32, device=device)
        actions = torch.tensor(actions, dtype=torch.long, device=device)
        rewards = torch.tensor(rewards, dtype=torch.float32, device=device)
        next_states = torch.tensor(np.stack(next_states), dtype=torch.float32, device=device)
        dones = torch.tensor(dones, dtype=torch.float32, device=device)
        return states, actions, rewards, next_states, dones

    def __len__(self):
        return len(self.buffer)


policy = DQN_CNN(state_dim, action_dim, lb, vars).to(device)
target = DQN_CNN(state_dim, action_dim, lb, vars).to(device)
target.load_state_dict(policy.state_dict())
optimizer = optim.Adam(policy.parameters(), lr=init_lr, weight_decay=1e-6)

replay = ReplayBuffer(buffer_capacity)


start = time.time()
total_rewards = []
valid_avg = []
losses = [] 
global_step = 0

for ep in range(max_episodes):
    e = time.time()
    state = env.reset(random_start=True, steps=max_steps)[0]
    state = norm(state)
    done = False
    ep_reward = 0

    for step in range(max_steps):
        global_step += 1

        # epsilon-greedy
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            with torch.no_grad():
                s = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
                q_vals = policy(s)  
                action = torch.argmax(q_vals, dim=1).item()

        next_state, reward, done, _, _ = env.step(action)
        next_state = norm(next_state)
        ep_reward += reward

        replay.push(state, action, reward, next_state, done)
        state = next_state

        # Sample and learn
        batch = replay.sample(batch_size, device)
        if batch is not None:
            states, actions, rewards, next_states, dones = batch

            # Current Q(s,a)
            q_pred = policy(states).gather(1, actions.unsqueeze(1)).squeeze(1)  

            # DDQN target:
            with torch.no_grad():
                next_q_policy = policy(next_states)                 
                next_actions = torch.argmax(next_q_policy, dim=1)   
                next_q_target = target(next_states)                 
                next_q = next_q_target.gather(1, next_actions.unsqueeze(1)).squeeze(1)  
                q_target = rewards + gamma * (1.0 - dones) * next_q

            
            loss = F.mse_loss(q_pred, q_target)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Periodic hard target update by steps
        if global_step % target_update_every == 0:
            target.load_state_dict(policy.state_dict())

        if done:
            break

    # epsilon decay per episode
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    total_rewards.append(ep_reward)

    #validation
    val_state = env_v2.reset(random_start=False)[0]
    val_state = norm(val_state)
    val_done = False
    val_ep_reward = 0
    while not val_done:
        with torch.no_grad():
            s = torch.tensor(val_state, dtype=torch.float32, device=device).unsqueeze(0)
            q_vals = policy(s)
            val_action = torch.argmax(q_vals, dim=1).item()
        val_next_state, val_reward, val_done, _, _ = env_v2.step(val_action)
        val_next_state = norm(val_next_state)
        val_ep_reward += val_reward
        val_state = val_next_state
    valid_avg.append(val_ep_reward)

    s = time.time()
    if ep % 10 == 0:
        print(f'Episode {ep}, Time {(s - e):.2f}s, '
              f'Train Reward: {ep_reward:.2f}, Avg Train: {np.mean(total_rewards):.2f}, '
              f'Val: {val_ep_reward:.2f}, Avg Val: {np.mean(valid_avg):.2f}')

    # Save every 25 episodes
    if (ep + 1) % 25 == 0:
        ckpt_path = f"ge_ddqn_policy_ep{ep+1:03d}.pth"
        torch.save(policy.state_dict(), ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

    if (ep + 1) % 25 == 0:
        fig, ax = plt.subplots(1, 2, figsize=(18, 4))
        ax[0].plot(total_rewards)
        ax[0].plot(pd.Series(total_rewards).rolling(30, min_periods=1).mean())
        ax[0].set_title('training reward')
        ax[1].plot(valid_avg)
        ax[1].plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean())
        ax[1].set_title('validation reward')
        plt.show()

end = time.time()
print('completed in', end - start)


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(valid_avg, label='valid reward ge no MPC', alpha=0.5)
plt.plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean(), label = 'valid smoothed reward', c='black', alpha=0.8)
plt.legend()
plt.xlabel('Episode Number', fontsize = 16)
plt.ylabel('Episode Reward', fontsize = 16)
plt.savefig(f'ge_no_mpc_valid_graph.png', dpi=1000)
plt.show()

In [ ]:
profit_ge2 = []
pr_ge2 = []
sr_ge2 = []
ar_ge2 = []
ports_ge2 = []
final_ge2 = []

for _ in range(10):
    # Reset environment
    state = env_v2.reset()[0]         
    state = norm(state)

    done = False
    episode_reward = 0
    episode_history = []

    while not done:
        with torch.no_grad():
            state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
            q_values = policy(state_tensor)              
            action = torch.argmax(q_values, dim=1).item()

        next_state, reward, done, _, info = env_v2.step(action)
        next_state = norm(next_state)

        episode_reward += reward
        episode_history.append(info)
        state = next_state
    
  
    pr_v_ge2 = pd.DataFrame(episode_history)
    port_ge2 = pr_v_ge2.Portfolio
    profit_g2 = profit_return(port_ge2)[0]
    pr_g2 = profit_return(port_ge2)[1]
    sr_g2 = sharpe_ratio(port_ge2)
    ar_g2 = annual_return(port_ge2)
    profit_ge2.append(profit_g2)
    pr_ge2.append(pr_g2)
    sr_ge2.append(sr_g2)
    ar_ge2.append(ar_g2)
    ports_ge2.append(port_ge2)
    final_ge2.append(port_ge2.iloc[-1])
    
bnhs_ge = (500000 * np.cumprod(1 + ge_valid[lb-1:].Close.pct_change().dropna()))
bnh_ge = bnhs_ge.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_ge-500000)
print('bnh sr',sharpe_ratio(bnhs_ge))
print('bnh ar', annual_return(bnhs_ge))

print('ge no MPC profit', calc_ci(profit_ge2))
print('ge no MPC pr', calc_ci(pr_ge2))
print('ge no MPC sr', calc_ci(sr_ge2))
print('ge no MPC ar', calc_ci(ar_ge2))

shaded_region(ports_ge2,bnhs_ge, final_ge2,'ge no MPC')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
ge_no_mpc_trial_runs = pd.DataFrame(ports_ge2).T
ge_no_mpc_trial_runs.to_csv('ge_no_mpc_trial_runs.csv')

### DJI no MPC

In [ ]:
env = envv2.SingleStockTraderKFohlcvCuilogr(dji_t_pre, lookback=lb, transaction =trans) 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import random
import numpy as np
import time

gamma = 0.95
action_dim = env.action_space.n
state_dim = lb * vars
buffer_capacity = 25000
batch_size = 8
max_episodes = 200
max_steps = 315
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
init_lr = 1e-3

epsilon = 1.0
epsilon_min = 0.1
epsilon_decay = 0.98

target_update_every = 2000  # steps

#Simple Replay Buffer
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        # store as numpy for memory efficiency
        self.buffer.append((
            np.array(state, dtype=np.float32),
            int(action),
            float(reward),
            np.array(next_state, dtype=np.float32),
            bool(done)
        ))

    def sample(self, batch_size, device):
        if len(self.buffer) < batch_size:
            return None
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.tensor(np.stack(states), dtype=torch.float32, device=device)
        actions = torch.tensor(actions, dtype=torch.long, device=device)
        rewards = torch.tensor(rewards, dtype=torch.float32, device=device)
        next_states = torch.tensor(np.stack(next_states), dtype=torch.float32, device=device)
        dones = torch.tensor(dones, dtype=torch.float32, device=device)
        return states, actions, rewards, next_states, dones

    def __len__(self):
        return len(self.buffer)


policy = DQN_CNN(state_dim, action_dim, lb, vars).to(device)
target = DQN_CNN(state_dim, action_dim, lb, vars).to(device)
target.load_state_dict(policy.state_dict())
optimizer = optim.Adam(policy.parameters(), lr=init_lr, weight_decay=1e-6)

replay = ReplayBuffer(buffer_capacity)


start = time.time()
total_rewards = []
valid_avg = []
losses = [] 
global_step = 0

for ep in range(max_episodes):
    e = time.time()
    state = env.reset(random_start=True, steps=max_steps)[0]
    state = norm(state)
    done = False
    ep_reward = 0

    for step in range(max_steps):
        global_step += 1

        # epsilon-greedy
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            with torch.no_grad():
                s = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
                q_vals = policy(s)  
                action = torch.argmax(q_vals, dim=1).item()

        next_state, reward, done, _, _ = env.step(action)
        next_state = norm(next_state)
        ep_reward += reward

        replay.push(state, action, reward, next_state, done)
        state = next_state

        # Sample and learn
        batch = replay.sample(batch_size, device)
        if batch is not None:
            states, actions, rewards, next_states, dones = batch

            # Current Q(s,a)
            q_pred = policy(states).gather(1, actions.unsqueeze(1)).squeeze(1)  

            # DDQN target:
            with torch.no_grad():
                next_q_policy = policy(next_states)                 
                next_actions = torch.argmax(next_q_policy, dim=1)   
                next_q_target = target(next_states)                 
                next_q = next_q_target.gather(1, next_actions.unsqueeze(1)).squeeze(1)  
                q_target = rewards + gamma * (1.0 - dones) * next_q

            
            loss = F.mse_loss(q_pred, q_target)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Periodic hard target update by steps
        if global_step % target_update_every == 0:
            target.load_state_dict(policy.state_dict())

        if done:
            break

    # epsilon decay per episode
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    total_rewards.append(ep_reward)

    #validation
    val_state = env_v3.reset(random_start=False)[0]
    val_state = norm(val_state)
    val_done = False
    val_ep_reward = 0
    while not val_done:
        with torch.no_grad():
            s = torch.tensor(val_state, dtype=torch.float32, device=device).unsqueeze(0)
            q_vals = policy(s)
            val_action = torch.argmax(q_vals, dim=1).item()
        val_next_state, val_reward, val_done, _, _ = env_v3.step(val_action)
        val_next_state = norm(val_next_state)
        val_ep_reward += val_reward
        val_state = val_next_state
    valid_avg.append(val_ep_reward)

    s = time.time()
    if ep % 10 == 0:
        print(f'Episode {ep}, Time {(s - e):.2f}s, '
              f'Train Reward: {ep_reward:.2f}, Avg Train: {np.mean(total_rewards):.2f}, '
              f'Val: {val_ep_reward:.2f}, Avg Val: {np.mean(valid_avg):.2f}')

    # Save every 25 episodes
    if (ep + 1) % 25 == 0:
        ckpt_path = f"dji_ddqn_policy_ep{ep+1:03d}.pth"
        torch.save(policy.state_dict(), ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

    if (ep + 1) % 25 == 0:
        fig, ax = plt.subplots(1, 2, figsize=(18, 4))
        ax[0].plot(total_rewards)
        ax[0].plot(pd.Series(total_rewards).rolling(30, min_periods=1).mean())
        ax[0].set_title('training reward')
        ax[1].plot(valid_avg)
        ax[1].plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean())
        ax[1].set_title('validation reward')
        plt.show()

end = time.time()
print('completed in', end - start)


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(valid_avg, label='valid reward dji no mpc', alpha=0.5)
plt.plot(pd.Series(valid_avg).rolling(30, min_periods=1).mean(), label = 'valid smoothed reward', c='black', alpha=0.8)
plt.legend()
plt.xlabel('Episode Number', fontsize = 16)
plt.ylabel('Episode Reward', fontsize = 16)
plt.savefig(f'dji_no_mpc_valid_graph.png', dpi=1000)
plt.show()

In [ ]:
profit_dji2 = []
pr_dji2 = []
sr_dji2 = []
ar_dji2 = []
ports_dji2 = []
final_dji2 = []

for _ in range(10):
    # Reset environment
    state = env_v3.reset()[0]         
    state = norm(state)

    done = False
    episode_reward = 0
    episode_history = []

    while not done:
        with torch.no_grad():
            state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
            q_values = policy(state_tensor)              
            action = torch.argmax(q_values, dim=1).item()

        next_state, reward, done, _, info = env_v3.step(action)
        next_state = norm(next_state)

        episode_reward += reward
        episode_history.append(info)
        state = next_state
  
    pr_v_dji2 = pd.DataFrame(episode_history)
    port_dji2 = pr_v_dji2.Portfolio
    profit_d2 = profit_return(port_dji2)[0]
    pr_d2 = profit_return(port_dji2)[1]
    sr_d2 = sharpe_ratio(port_dji2)
    ar_d2 = annual_return(port_dji2)
    profit_dji2.append(profit_d2)
    pr_dji2.append(pr_d2)
    sr_dji2.append(sr_d2)
    ar_dji2.append(ar_d2)
    ports_dji2.append(port_dji2)
    final_dji2.append(port_dji2.iloc[-1])
    
bnhs_dji = (500000 * np.cumprod(1 + dji_valid[lb-1:].Close.pct_change().dropna()))
bnh_dji = bnhs_dji.iloc[-1]

#calculate the CI for all evulation metrics 
print('bnh profit', bnh_dji-500000)
print('bnh sr',sharpe_ratio(bnhs_dji))
print('bnh ar', annual_return(bnhs_dji))

print('dji no MPC profit', calc_ci(profit_dji2))
print('dji no MPC pr', calc_ci(pr_dji2))
print('dji no MPC sr', calc_ci(sr_dji2))
print('dji no MPC ar', calc_ci(ar_dji2))
shaded_region(ports_dji2,bnhs_dji,final_dji2,'dji no MPC')

In [ ]:
#save the ind runs if needed for future work or graph tweaking
dji_no_mpc_trial_runs = pd.DataFrame(ports_dji2).T
dji_no_mpc_trial_runs.to_csv('dji_no_mpc_trial_runs.csv')